<a href="https://colab.research.google.com/github/MatienkoAndrew/notebooks/blob/main/andrey_karpathy_gpt_dev.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# FUNC

In [1]:
import os

def save(bigram_code: str, FILENAME: str):
    with open(FILENAME, 'w') as file:
        file.write(bigram_code)
        print(f"Saved file at: {os.path.abspath(FILENAME)}")

# Download file

In [2]:
# ! wget https://github.com/karpathy/ng-video-lecture/blob/master/input.txt
! wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

--2023-03-17 16:00:48--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘input.txt’

input.txt           100%[===================>]   1.06M  --.-KB/s    in 0.02s   

2023-03-17 16:00:48 (55.4 MB/s) - ‘input.txt’ saved [1115394/1115394]



# Read file

In [3]:
# read it to inspect it
with open('input.txt', 'r', encoding='utf-8') as file:
    text = file.read()

In [4]:
print(f"length of dataset in characters is {len(text)}")

length of dataset in characters is 1115394


In [5]:
##-- first 1000 chatacters
print(text[:1000])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor citizens, the patricians good.
What authority surfeits on would relieve us: if they
would yield us but the superfluity, while it were
wholesome, we might guess they relieved us humanely;
but they think we are too dear: the leanness that
afflicts us, the object of our misery, is as an
inventory to particularise their abundance; our
sufferance is a gain to them Let us revenge this with
our pikes, ere we become rakes: for the gods know I
speak this in hunger for bread, not in thirst for revenge.



## all the unique character that occur in this text

In [6]:
# here all the unique character that occur in this text
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


## create mapping from characters to integers

In [7]:
##-- create mapping from characters to integers
stoi = { ch: i for i, ch in enumerate(chars) }
atoi = { i: ch for i, ch in enumerate(chars) }

encode = lambda s: [stoi[c] for c in s]
decode = lambda l: ''.join([atoi[i] for i in l])
print(encode("hi there")) ##-- encoder: take a string, output a list of integers
print(decode(encode('hi there'))) ##-- decoder: take a list of integers, output a string

[46, 47, 1, 58, 46, 43, 56, 43]
hi there


## encode the entire text dataset

In [8]:
##-- let's encode the entire text dataset and store it into a torch.Tensor
import torch
data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype)
print(data[:1000])

torch.Size([1115394]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59,  1, 39, 56, 43,  1, 39, 50, 50,
         1, 56, 43, 57, 53, 50, 60, 43, 42,  1, 56, 39, 58, 46, 43, 56,  1, 58,
        53,  1, 42, 47, 43,  1, 58, 46, 39, 52,  1, 58, 53,  1, 44, 39, 51, 47,
        57, 46, 12,  0,  0, 13, 50, 50, 10,  0, 30, 43, 57, 53, 50, 60, 43, 42,
         8,  1, 56, 43, 57, 53, 50, 60, 43, 42,  8,  0,  0, 18, 47, 56, 57, 58,
         1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 18, 47, 56, 57, 58,  6,  1, 63,
        53, 59,  1, 49, 52, 53, 61,  1, 15, 39, 47, 59, 57,  1, 25, 39, 56, 41,
      

## train_test_split

In [9]:
##-- train test split
n = int(0.9 * len(data))
train_data = data[:n]
val_data = data[n:]
print(f"len(train_data)={len(train_data)}")
print(f"len(val_data)={len(val_data)}")

len(train_data)=1003854
len(val_data)=111540


## Block size

Модель обучается не на всем блоке данных (это было бы слишком трудозатратно), а на частях данных (чанках)

Возьмем размер блока в 8 символов. На самом деле, в этот блок "вшито" 8 примеров.

Трансформер учится делать предсказание для каждой позиции (2 ячейка)

In [10]:
block_size = 8
train_data[:block_size+1]

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

In [11]:
x = train_data[:block_size]
y = train_data[1: block_size+1]
for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print(f"when input is {context} the target: {target}")

when input is tensor([18]) the target: 47
when input is tensor([18, 47]) the target: 56
when input is tensor([18, 47, 56]) the target: 57
when input is tensor([18, 47, 56, 57]) the target: 58
when input is tensor([18, 47, 56, 57, 58]) the target: 1
when input is tensor([18, 47, 56, 57, 58,  1]) the target: 15
when input is tensor([18, 47, 56, 57, 58,  1, 15]) the target: 47
when input is tensor([18, 47, 56, 57, 58,  1, 15, 47]) the target: 58


У нас есть одно измерение - поданное предложение

Есть еще одно - батч

В модель подается партия предложений (батч), которая складывается в один тензор (для ускорения работы)


In [12]:
torch.manual_seed(1337)
batch_size = 4 ##-- сколько независимых объектов (предложений) обрабатывается за раз (параллельно)
block_size = 8 ##-- максимальная длина предложения (в нашем случае, количество символов)

def get_batch(split: str='train'):
    '''
    generate a small batch of data of inputs x and targets y

    split: str: train or test
    '''
    data = train_data if split=='train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size, )) ##-- random idx
    x = torch.stack([data[i: i+block_size] for i in ix])
    y = torch.stack([data[i+1: i+block_size+1] for i in ix])
    return (x, y)

xb, yb = get_batch('train')
print('inputs:')
print(xb.shape)
print(xb)
print('targets:')
print(yb.shape)
print(yb)

print('----')

for b in range(batch_size):
    for t in range(block_size):
        context = xb[b, :t+1]
        target = yb[b, t]
        print(f"when input is {context.tolist()} the target: {target}")

inputs:
torch.Size([4, 8])
tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])
targets:
torch.Size([4, 8])
tensor([[43, 58,  5, 57,  1, 46, 43, 39],
        [53, 56,  1, 58, 46, 39, 58,  1],
        [58,  1, 58, 46, 39, 58,  1, 46],
        [17, 27, 10,  0, 21,  1, 54, 39]])
----
when input is [24] the target: 43
when input is [24, 43] the target: 58
when input is [24, 43, 58] the target: 5
when input is [24, 43, 58, 5] the target: 57
when input is [24, 43, 58, 5, 57] the target: 1
when input is [24, 43, 58, 5, 57, 1] the target: 46
when input is [24, 43, 58, 5, 57, 1, 46] the target: 43
when input is [24, 43, 58, 5, 57, 1, 46, 43] the target: 39
when input is [44] the target: 53
when input is [44, 53] the target: 56
when input is [44, 53, 56] the target: 1
when input is [44, 53, 56, 1] the target: 58
when input is [44, 53, 56, 1, 58] the target: 46
when input is [44, 53

# BigramLanguageModel

In [13]:
import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)

class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        ##-- each token directly read oof the logits for the next token from a lookip table
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, idx, targets=None):
        ##-- idx and targets are both (B, T) tensor of integers
        logits = self.token_embedding_table(idx) # (B, T, C) #-- (4, 8, 65), (batch_size, block_size, vocab_size)
        # print(logits.shape) # (4, 8, 65)
        # print(targets.shape) # (4, 8)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            ##-- из батча размером 4 предложения переводим в одну строку (то есть четыре предложения в одно)
            logits = logits.view(B*T, C) ## (32, 65)
            ##-- второй вариант, таргеты в таком случае тоже не менять
            # logits = torch.permute(logits, (0, 2, 1))
            targets = targets.view(B*T) ## (32)
            loss = F.cross_entropy(logits, targets)
        return logits, loss

    def generate(self, idx, max_new_tokens: int=100):
        ##-- idx is the (B, T) array of the indices in the current context
        for _ in range(max_new_tokens):
            ##-- get the predictions
            logits, _ = self(idx)
            ##-- focus only on the last time step (последнюю букву)
            # print(f"logits_before ={logits}, {logits.shape}")
            logits = logits[:, -1, :] # becomes (B, C)
            # print(f"logits_after ={logits}, {logits.shape}")
            ##-- softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # print(probs)
            ##-- sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # print(idx_next)
            ##-- append sampled idx to the running sample
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

m = BigramLanguageModel(vocab_size)
logits, loss = m(xb, yb)
print(logits.shape)
print(loss)


# idx = torch.zeros((2, 1), dtype=torch.long)
# print(idx)
# m_gen = m.generate(idx, 100)
# print(m_gen)
# print(decode(m_gen[0].tolist()))
# print(decode(m_gen))

torch.Size([32, 65])
tensor(4.8786, grad_fn=<NllLossBackward0>)


## optimizer

In [14]:
##-- pytorch optimizer
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3)

In [15]:
batch_size = 32
for steps in range(1000):
    ##-- sample a batch of data
    xb, yb = get_batch('train')
    ##-- evaluate the loss
    logits, loss = m(xb, yb)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
print(loss.item())

3.806382894515991


In [16]:
idx = torch.zeros((2, 1), dtype=torch.long)
m_gen = m.generate(idx, 300)
# print(m_gen)
print(decode(m_gen[0].tolist()))


SK$G?wks;d F!WhP,SlRxjXy srTrjRjJMNAYdYlviOnQ!zEXXg;Sissw
AkgjXEr&y,jKusuDHkzhSBibJUFlTCATErgCOQoCM$-wowaythjKby:CHpxQcma-b&yUzoaHo-IfqZIotiGjKApA?$gYn&QW fuky-MsycxqSl,
Sv?q$sq,pyh rnQxyk
SmJKHASxf?,susERn3d fmZMpoC-EZ-IVwF-pZMF.?oI'XjJXEr&!
SZa faor$VqolmfBjUVRyoFo?helxjgmJHof bpT di;vW:CVJ?nQbEe,


In [17]:
#@title bigram.py
bigram_script = \
"""
import torch
import torch.nn as nn
from torch.nn import functional as F

##-- hyperparameters
batch_size = 32 ##-- сколько независимых объектов (предложений) обрабатывается за раз (параллельно)
block_size = 8 ##-- максимальная длина предложения (в нашем случае, количество символов)
max_iters = 3000
eval_interval = 300
learning_rate = 1e-2
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200
##------------------

torch.manual_seed(1337)

# wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
with open('input.txt', 'r', encoding='utf-8') as file:
    text = file.read()

# here all the unique character that occur in this text
chars = sorted(list(set(text)))
vocab_size = len(chars)
##-- create mapping from characters to integers
stoi = { ch: i for i, ch in enumerate(chars) }
atoi = { i: ch for i, ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s]
decode = lambda l: ''.join([atoi[i] for i in l])

##-- Train and test splits
data = torch.tensor(encode(text), dtype=torch.long)
n = int(0.9 * len(data))
train_data = data[:n]
val_data = data[n:]

##-- data loading
def get_batch(split: str='train'):
    '''
    generate a small batch of data of inputs x and targets y

    split: str: train or test
    '''
    data = train_data if split=='train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size, )) ##-- random idx
    x = torch.stack([data[i: i+block_size] for i in ix])
    y = torch.stack([data[i+1: i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return (x, y)

@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out


##-- super simple bigram model
class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        ##-- each token directly read oof the logits for the next token from a lookip table
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, idx, targets=None):
        ##-- idx and targets are both (B, T) tensor of integers
        logits = self.token_embedding_table(idx) # (B, T, C) #-- (4, 8, 65), (batch_size, block_size, vocab_size)
        # print(logits.shape) # (4, 8, 65)
        # print(targets.shape) # (4, 8)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            ##-- из батча размером 4 предложения переводим в одну строку (то есть четыре предложения в одно)
            logits = logits.view(B*T, C)
            ##-- второй вариант, таргеты в таком случае тоже не менять
            # logits = torch.permute(logits, (0, 2, 1))
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
        return logits, loss

    def generate(self, idx, max_new_tokens: int=100):
        ##-- idx is the (B, T) array of the indices in the current context
        for _ in range(max_new_tokens):
            ##-- get the predictions
            logits, _ = self(idx)
            ##-- focus only on the last time step (последнюю букву)
            # print(f"logits_before ={logits}, {logits.shape}")
            logits = logits[:, -1, :] # becomes (B, C)
            # print(f"logits_after ={logits}, {logits.shape}")
            ##-- softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # print(probs)
            ##-- sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # print(idx_next)
            ##-- append sampled idx to the running sample
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

model = BigramLanguageModel(vocab_size)
m = model.to(device)

##-- pytorch optimizer
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3)

##-- fit

for iter in range(max_iters):
    if iter % eval_interval == 0:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    ##-- sample a batch of data
    xb, yb = get_batch('train')

    ##-- evaluate the loss
    logits, loss = m(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

##-- generate the model
context = torch.zeros((1, 1), dtype=torch.long, device=device)
print(decode(m.generate(context, max_new_tokens=500)[0].tolist()))
"""

In [18]:
FILENAME = 'bigram.py'
save(bigram_script, FILENAME)

Saved file at: /content/bigram.py


In [19]:
%%time

! python bigram.py

step 0: train loss 4.7305, val loss 4.7241
step 300: train loss 4.3818, val loss 4.3896
step 600: train loss 4.0801, val loss 4.0784
step 900: train loss 3.8066, val loss 3.8117
step 1200: train loss 3.5844, val loss 3.5850
step 1500: train loss 3.3757, val loss 3.3829
step 1800: train loss 3.2182, val loss 3.2218
step 2100: train loss 3.0817, val loss 3.0810
step 2400: train loss 2.9663, val loss 2.9739
step 2700: train loss 2.8809, val loss 2.8800



CExfikRO:
wcowi,STAOLOL, btK

HAPTombobeAUGe.
SGJO-33SM:C?YIUauss:LVXEthafNusqhathe.t?ar dXlaSpates wicrd RWI,
DERacomzoroup
Yow&$FMOUf isth bHEv!$Whedillxcaeg ireeYERngmin latiHNGAdrov ts, anenWk p.
GRWilyWjbo!
el.lind me u.
-huD3SPy wiry:CUEOKMORT'X3Qw y. w'sBHUSInormopeYelgCIEJMk:
Gll, d motSPkllo W-woo whrVCeiib3s wor m dE$HZAETENGShireAs p-LK3:Cl-xTre

ALkOMmnterupt f s z; iris!
m:CENGjey aleUE$ERUNMadPrD?d KISo myaHKINLIk!
Ktiyb&y,:
SadaplWPT:VE:zLUYBinin cNuk?ayeaney Iry tsmI&fy VEc!3My
CPU times: user 107 ms, sys: 18.1 ms, total:

# Self-attention

Мы хотим, чтобы 5 токен узнал о 4-ем, третьем, втором, первом и нулевом токенах. Но не знал о шестом и так далее. Потому что мы не заглядываем в будущее

Какой самый простой способ дать пятому токену информацию о предыдущих четырех?

Усреднить предыдущие четыре (и наш пятый) - это и будет информация о нас и предыдущих токенах (истории).

P.S. Усреднение - очень простой и слабый способ передачи информации, так как теряется очень много информации, но сейчас для практики - норм

----
Можно использовать взвешенную агрегацию предыдущих элементов, используя матричное перемножение

In [20]:
torch.manual_seed(1337)

B, T, C = (4, 8, 2)
x = torch.randn((B, T, C))
x.shape

torch.Size([4, 8, 2])

### Loop

In [21]:
##-- version 1: use loop
# we want x[b, t] = mean_{i<=t} x[b, i]
xbow = torch.zeros((B, T, C))
for b in range(B):
    for t in range(T):
        xprev = x[b, :t+1] # (t, C)
        xbow[b, t] = torch.mean(xprev, dim=0)

In [22]:
x

tensor([[[ 0.1808, -0.0700],
         [-0.3596, -0.9152],
         [ 0.6258,  0.0255],
         [ 0.9545,  0.0643],
         [ 0.3612,  1.1679],
         [-1.3499, -0.5102],
         [ 0.2360, -0.2398],
         [-0.9211,  1.5433]],

        [[ 1.3488, -0.1396],
         [ 0.2858,  0.9651],
         [-2.0371,  0.4931],
         [ 1.4870,  0.5910],
         [ 0.1260, -1.5627],
         [-1.1601, -0.3348],
         [ 0.4478, -0.8016],
         [ 1.5236,  2.5086]],

        [[-0.6631, -0.2513],
         [ 1.0101,  0.1215],
         [ 0.1584,  1.1340],
         [-1.1539, -0.2984],
         [-0.5075, -0.9239],
         [ 0.5467, -1.4948],
         [-1.2057,  0.5718],
         [-0.5974, -0.6937]],

        [[ 1.6455, -0.8030],
         [ 1.3514, -0.2759],
         [-1.5108,  2.1048],
         [ 2.7630, -1.7465],
         [ 1.4516, -1.5103],
         [ 0.8212, -0.2115],
         [ 0.7789,  1.5333],
         [ 1.6097, -0.4032]]])

In [23]:
xbow

tensor([[[ 0.1808, -0.0700],
         [-0.0894, -0.4926],
         [ 0.1490, -0.3199],
         [ 0.3504, -0.2238],
         [ 0.3525,  0.0545],
         [ 0.0688, -0.0396],
         [ 0.0927, -0.0682],
         [-0.0341,  0.1332]],

        [[ 1.3488, -0.1396],
         [ 0.8173,  0.4127],
         [-0.1342,  0.4395],
         [ 0.2711,  0.4774],
         [ 0.2421,  0.0694],
         [ 0.0084,  0.0020],
         [ 0.0712, -0.1128],
         [ 0.2527,  0.2149]],

        [[-0.6631, -0.2513],
         [ 0.1735, -0.0649],
         [ 0.1685,  0.3348],
         [-0.1621,  0.1765],
         [-0.2312, -0.0436],
         [-0.1015, -0.2855],
         [-0.2593, -0.1630],
         [-0.3015, -0.2293]],

        [[ 1.6455, -0.8030],
         [ 1.4985, -0.5395],
         [ 0.4954,  0.3420],
         [ 1.0623, -0.1802],
         [ 1.1401, -0.4462],
         [ 1.0870, -0.4071],
         [ 1.0430, -0.1299],
         [ 1.1138, -0.1641]]])

In [24]:
x

tensor([[[ 0.1808, -0.0700],
         [-0.3596, -0.9152],
         [ 0.6258,  0.0255],
         [ 0.9545,  0.0643],
         [ 0.3612,  1.1679],
         [-1.3499, -0.5102],
         [ 0.2360, -0.2398],
         [-0.9211,  1.5433]],

        [[ 1.3488, -0.1396],
         [ 0.2858,  0.9651],
         [-2.0371,  0.4931],
         [ 1.4870,  0.5910],
         [ 0.1260, -1.5627],
         [-1.1601, -0.3348],
         [ 0.4478, -0.8016],
         [ 1.5236,  2.5086]],

        [[-0.6631, -0.2513],
         [ 1.0101,  0.1215],
         [ 0.1584,  1.1340],
         [-1.1539, -0.2984],
         [-0.5075, -0.9239],
         [ 0.5467, -1.4948],
         [-1.2057,  0.5718],
         [-0.5974, -0.6937]],

        [[ 1.6455, -0.8030],
         [ 1.3514, -0.2759],
         [-1.5108,  2.1048],
         [ 2.7630, -1.7465],
         [ 1.4516, -1.5103],
         [ 0.8212, -0.2115],
         [ 0.7789,  1.5333],
         [ 1.6097, -0.4032]]])

In [25]:
torch.manual_seed(42)
a = torch.ones(3, 3)
b = torch.randint(0, 10, (3, 2)).float()
c = a @ b
print('a=')
print(a)
print('--')
print('b=')
print(b)
print('--')
print('c=')
print(c)

a=
tensor([[1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.]])
--
b=
tensor([[2., 7.],
        [6., 4.],
        [6., 5.]])
--
c=
tensor([[14., 16.],
        [14., 16.],
        [14., 16.]])


In [26]:
##-- трюк - как проссумировать строки матричным перемножением
torch.manual_seed(42)
a = torch.tril(torch.ones(3, 3))
b = torch.randint(0, 10, (3, 2)).float()
c = a @ b
print('a=')
print(a)
print('--')
print('b=')
print(b)
print('--')
print('c=')
print(c)

a=
tensor([[1., 0., 0.],
        [1., 1., 0.],
        [1., 1., 1.]])
--
b=
tensor([[2., 7.],
        [6., 4.],
        [6., 5.]])
--
c=
tensor([[ 2.,  7.],
        [ 8., 11.],
        [14., 16.]])


In [27]:
##-- трюк - как получить среднее матричным перемножением
torch.manual_seed(42)
a = torch.tril(torch.ones(3, 3))
sum_ = torch.sum(a, 1, keepdim=True)
print(f"sum_={sum_}")
a = a / sum_
b = torch.randint(0, 10, (3, 2)).float()
c = a @ b
print('a=')
print(a)
print('--')
print('b=')
print(b)
print('--')
print('c=')
print(c)

sum_=tensor([[1.],
        [2.],
        [3.]])
a=
tensor([[1.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000],
        [0.3333, 0.3333, 0.3333]])
--
b=
tensor([[2., 7.],
        [6., 4.],
        [6., 5.]])
--
c=
tensor([[2.0000, 7.0000],
        [4.0000, 5.5000],
        [4.6667, 5.3333]])


### Matrix

In [28]:
##-- version 2: use matrix
wei = torch.tril(torch.ones((T, T)))
wei = wei / wei.sum(1, keepdim=True)
xbow2 = wei @ x # (B, T, T) @ (B, T, C) ----> (B, T, C)

In [29]:
wei

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3333, 0.3333, 0.3333, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2500, 0.2500, 0.2500, 0.2500, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2000, 0.2000, 0.2000, 0.2000, 0.2000, 0.0000, 0.0000, 0.0000],
        [0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.0000, 0.0000],
        [0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.0000],
        [0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250]])

In [30]:
xbow2

tensor([[[ 0.1808, -0.0700],
         [-0.0894, -0.4926],
         [ 0.1490, -0.3199],
         [ 0.3504, -0.2238],
         [ 0.3525,  0.0545],
         [ 0.0688, -0.0396],
         [ 0.0927, -0.0682],
         [-0.0341,  0.1332]],

        [[ 1.3488, -0.1396],
         [ 0.8173,  0.4127],
         [-0.1342,  0.4395],
         [ 0.2711,  0.4774],
         [ 0.2421,  0.0694],
         [ 0.0084,  0.0020],
         [ 0.0712, -0.1128],
         [ 0.2527,  0.2149]],

        [[-0.6631, -0.2513],
         [ 0.1735, -0.0649],
         [ 0.1685,  0.3348],
         [-0.1621,  0.1765],
         [-0.2312, -0.0436],
         [-0.1015, -0.2855],
         [-0.2593, -0.1630],
         [-0.3015, -0.2293]],

        [[ 1.6455, -0.8030],
         [ 1.4985, -0.5395],
         [ 0.4954,  0.3420],
         [ 1.0623, -0.1802],
         [ 1.1401, -0.4462],
         [ 1.0870, -0.4071],
         [ 1.0430, -0.1299],
         [ 1.1138, -0.1641]]])

In [31]:
##-- матричным перемножением получаем такой же результат
torch.allclose(xbow, xbow2)

True

### Use softmax

In [32]:
##-- version 3: use Softmax
tril = torch.tril(torch.ones(T, T)) ##-- сколько элементов агрегируется
print(f"tril={tril}")
print('----')

wei = torch.zeros((T, T))
print(f"wei={wei}")
print('----')

wei = wei.masked_fill(tril==0, float('-inf'))
print(f"wei={wei}")
print('----')

wei = F.softmax(wei, dim=-1)
print(f"wei={wei}")
print('----')

xbow3 = wei @ x
print(f"xbow3={xbow3}")
print('----')
torch.allclose(xbow, xbow3)

tril=tensor([[1., 0., 0., 0., 0., 0., 0., 0.],
        [1., 1., 0., 0., 0., 0., 0., 0.],
        [1., 1., 1., 0., 0., 0., 0., 0.],
        [1., 1., 1., 1., 0., 0., 0., 0.],
        [1., 1., 1., 1., 1., 0., 0., 0.],
        [1., 1., 1., 1., 1., 1., 0., 0.],
        [1., 1., 1., 1., 1., 1., 1., 0.],
        [1., 1., 1., 1., 1., 1., 1., 1.]])
----
wei=tensor([[0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.]])
----
wei=tensor([[0., -inf, -inf, -inf, -inf, -inf, -inf, -inf],
        [0., 0., -inf, -inf, -inf, -inf, -inf, -inf],
        [0., 0., 0., -inf, -inf, -inf, -inf, -inf],
        [0., 0., 0., 0., -inf, -inf, -inf, -inf],
        [0., 0., 0., 0., 0., -inf, -inf, -inf],
        [0., 0., 0., 0., 0., 0., -inf, -inf

True

In [33]:
#@title bigram_v2.py
bigram_v2 = \
"""

import torch
import torch.nn as nn
from torch.nn import functional as F

##-- hyperparameters
batch_size = 4 ##-- сколько независимых объектов (предложений) обрабатывается за раз (параллельно)
block_size = 8 ##-- максимальная длина предложения (в нашем случае, количество символов)
max_iters = 300
eval_interval = 300
learning_rate = 1e-2
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200
n_embd = 32 ##-- размер эмбединга
##------------------

torch.manual_seed(1337)

# wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
with open('input.txt', 'r', encoding='utf-8') as file:
    text = file.read()

# here all the unique character that occur in this text
chars = sorted(list(set(text)))
vocab_size = len(chars)
##-- create mapping from characters to integers
stoi = { ch: i for i, ch in enumerate(chars) }
atoi = { i: ch for i, ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s]
decode = lambda l: ''.join([atoi[i] for i in l])

##-- Train and test splits
data = torch.tensor(encode(text), dtype=torch.long)
n = int(0.9 * len(data))
train_data = data[:n]
val_data = data[n:]

##-- data loading
def get_batch(split: str='train'):
    '''
    generate a small batch of data of inputs x and targets y

    split: str: train or test
    '''
    data = train_data if split=='train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size, )) ##-- random idx
    x = torch.stack([data[i: i+block_size] for i in ix])
    y = torch.stack([data[i+1: i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return (x, y)

@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out


##-- super simple bigram model
class BigramLanguageModel(nn.Module):
    def __init__(self):
        super().__init__()
        ##-- each token directly read oof the logits for the next token from a lookip table
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd) ##-- берем эмбединг токена (кодируем токен)
        self.position_embedding_table = nn.Embedding(block_size, n_embd) ##-- кодируем позицию токена
        self.lm_head = nn.Linear(n_embd, vocab_size) ##-- linear_model head

    def forward(self, idx, targets=None):
        B, T = idx.shape

        ##-- idx and targets are both (B, T) tensor of integers
        tok_emb = self.token_embedding_table(idx) # (B, T, C) #-- (4, 8, 32), (batch_size, block_size, emb_size)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device)) # (T, C) (8, 32) ##-- кодируем позицию токена
        x = tok_emb + pos_emb ##-- (B, T, C) эмбединг токена и его позиции
        logits = self.lm_head(x) # (B, T, vocab_size) #-- (4, 8, 65), (batch_size, block_size, vocab_size)
        # print(logits.shape) # (4, 8, 65)
        # print(targets.shape) # (4, 8)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            ##-- из батча размером 4 предложения переводим в одну строку (то есть четыре предложения в одно)
            logits = logits.view(B*T, C)
            ##-- второй вариант, таргеты в таком случае тоже не менять
            # logits = torch.permute(logits, (0, 2, 1))
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
        return logits, loss

    def generate(self, idx, max_new_tokens: int=100):
        ##-- idx is the (B, T) array of the indices in the current context
        for _ in range(max_new_tokens):
            ##-- get the predictions
            logits, _ = self(idx)
            ##-- focus only on the last time step (последнюю букву)
            print(f"logits_before ={logits}, {logits.shape}")
            logits = logits[:, -1, :] # becomes (B, C)
            print(f"logits_after ={logits}, {logits.shape}")
            ##-- softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            print(probs)
            ##-- sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            print(idx_next)
            ##-- append sampled idx to the running sample
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

model = BigramLanguageModel()
m = model.to(device)

##-- pytorch optimizer
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3)

##-- fit

for iter in range(max_iters):
    if iter % eval_interval == 0:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    ##-- sample a batch of data
    xb, yb = get_batch('train')

    ##-- evaluate the loss
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

##-- generate from the model
context = torch.zeros((1, 1), dtype=torch.long, device=device)
print(decode(m.generate(context, max_new_tokens=10)[0].tolist()))

"""

In [34]:
FILENAME = 'bigram_v2.py'
save(bigram_v2, FILENAME)

Saved file at: /content/bigram_v2.py


In [35]:
! diff bigram.py bigram_v2.py

1a2
> 
7c8
< batch_size = 32 ##-- сколько независимых объектов (предложений) обрабатывается за раз (параллельно)
---
> batch_size = 4 ##-- сколько независимых объектов (предложений) обрабатывается за раз (параллельно)
9c10
< max_iters = 3000
---
> max_iters = 300
13a15
> n_embd = 32 ##-- размер эмбединга
68c70
<     def __init__(self, vocab_size):
---
>     def __init__(self):
71c73,75
<         self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)
---
>         self.token_embedding_table = nn.Embedding(vocab_size, n_embd) ##-- берем эмбединг токена (кодируем токен)
>         self.position_embedding_table = nn.Embedding(block_size, n_embd) ##-- кодируем позицию токена
>         self.lm_head = nn.Linear(n_embd, vocab_size) ##-- linear_model head
73a78,79
>         B, T = idx.shape
> 
75c81,84
<         logits = self.token_embedding_table(idx) # (B, T, C) #-- (4, 8, 65), (batch_size, block_size, vocab_size)
---
>         tok_emb = self.token_embedding_table(idx) # (B, T, C) #

In [36]:
%%time

! python bigram_v2.py

step 0: train loss 4.4701, val loss 4.4930
logits_before =tensor([[[ 0.1875,  0.2098, -1.2989, -0.7939, -0.6377, -0.5834, -0.1427,
          -1.3647, -0.8279, -0.9151, -0.7947, -1.2249, -0.4032,  0.4988,
          -0.2799,  0.0588,  0.0020, -0.7513, -0.9078, -0.9270,  0.2285,
          -0.6461, -0.6058, -0.5217, -0.5360,  0.4208, -0.5117, -0.0905,
          -0.2599, -0.5530,  0.3508,  1.0032,  0.1444, -0.4868, -0.4084,
          -0.1927, -1.1498, -1.0945, -0.6877,  0.3868, -0.0950,  0.1140,
          -0.3839, -0.2191,  0.1055,  0.3010, -0.7910, -0.5693, -0.4552,
          -0.1785, -0.3584,  0.4609,  0.9867, -0.2026, -0.3343, -0.4283,
           0.8460,  0.1427,  0.0605, -1.2164, -0.0200, -0.0623, -0.9823,
          -0.1947, -0.5709]]], device='cuda:0', grad_fn=<ViewBackward0>), torch.Size([1, 1, 65])
logits_after =tensor([[ 0.1875,  0.2098, -1.2989, -0.7939, -0.6377, -0.5834, -0.1427, -1.3647,
         -0.8279, -0.9151, -0.7947, -1.2249, -0.4032,  0.4988, -0.2799,  0.0588,
          0.

## self-attention implementation

`wei = torch.zeros((T, T))` - данный шаг создает равномерное распределение важности токенов. Нам это неинтересно. Нам бы лучше знать, что некоторые токены имеют больший вес. Например, если сейчас мы на гласной, то ранее нам более интересны согласные.

В общем, нужно для каждого токена назначить вес. И получить взвешенное среднее. Данную проблему решает *self-attention*

*Как self-attention решает данную проблему?*
---
1. Каждый токен будет иметь две матрицы:

    a. query - отвечает на вопрос "Что я ищу?"

    b. key - отвечает на вопрос "Что я содержу?"


И чтобы узнать взаимоотношения между векторами - нужно перемножить все запросы со всеми ключами. (то есть 8 запросов перемножаются с 8 ключами, 64 перемножения).

*Запрос перемножается с каждым ключом и данное перемножение становится взвешенным средним (то есть мы для каждого токена назначаем вес, насколько этот токен важен для текущего токена?)*


Также у нас есть еще одно значение, которое называется ЗНАЧЕНИЕ *value*

`x` - это личная информация токена

`value` - говорит о том, что я содержу, если я вам интересен (после перемножения query @ key)

In [37]:
##-- version4: self-attention

torch.manual_seed(1337)
B, T, C = 4, 8, 2 # batch, time, channels
x = torch.randn((B, T, C))

##-- single head in self-attention
head_size = 16 # 2 ##-- hyperparameter
query = nn.Linear(C, head_size, bias=False) ## веса случайные # key.weight
key = nn.Linear(C, head_size, bias=False) ## другие веса случайные # query.weight
value = nn.Linear(C, head_size, bias=False) ## другие веса случайные # value.weight
# print(f"query.weight={query.weight}")
# print(f"key.weight={key.weight}")
##-- для каждого токена получаем запрос и ключ (пока никакой коммуникации)
q = query(x) # (B, T, 16) ##-- матричное перемножение с фиксированными весами (1x32 --> 1x16)
k = key(x) # (B, T, 16) ##-- матричное перемножение с фиксированными весами (1x32 --> 1x16)
# print(f"q={q}")
# print(f"k={k}")
##-- коммуникация между токенами (query @ key)
wei = q @ k.transpose(-2, -1) ##-- (B, T, 16) x (B, 16, T) -->> (B, T, T)
# print(f"wei[0]={wei[0]}")


##-- усреднение предыдущих токенов (и текущего)
tril = torch.tril(torch.ones((T, T)))
# wei = torch.zeros((T, T))
wei = wei.masked_fill(tril==0, float('-inf'))
wei = F.softmax(wei, dim=-1) # (B, T, T)

##-- value
v = value(x) # (B, T, 16) ##-- матричное перемножение с фиксированными весами (1x32 --> 1x16)
out = wei @ v # (B, T, T) @ (B, T, 16) -->> (B, T, 16)

out.shape

torch.Size([4, 8, 16])

- Notes

1. Attention - это механизм внимания. В нашем случае, первый токен соединен с самим собой. Второй токен - с самим собой и с первым токеном. Третий - с собой, вторым и третим. И так далее.

2. Нет никакого понятия пространства. Первоначально, когда мы задаем токены, они не знают ничего друг о друге и где они находятся. Чтобы они знали - нужно добавить attention - веса для каждого токена (переменная `wei`)

3. Батчи не разговаривают друг с другом. То есть у нас есть батч размера 4 и получается, что третий батч не общается ни со вторым батчем, ни с четвертым. Батчи обрабатываются независимо

4. В структуре энкодера данная строка (обычно) удаляется `wei = wei.masked_fill(tril==0, float('-inf'))` - так мы разрешаем первому токену смотреть на последний. То есть все токены видят друг друга.

    Но в декодере данная строка присутствует, чтобы не заглядывать в будущее.

5. Помимо *self-attention* существует *cross-attention*.

    В self-attention *query, key, value* приходят из одного источника `X`. То есть `X` создает *query, key, value*. То есть токены разговаривают друг с другом.

    В cross-attention *query* приходят из одного источника `X`, а *key, value* приходят из доп.источника. То есть `X` создает *query*. А *key, value* создаются, например, из энкодера. То есть создается запрос и ищется ответ из другого источника.

6. Нужно нормализвать dot.product(query, key). Иначе дисперсия будет большой. Если дисперсия большая, то softmax ломается. Он подстраивается под самое большое значение. И вероятности становятся нулями и одна единица (других вариантов нет), что плохо.

In [38]:
##-- пример нормализации весов, чтобы уменьшить дисперсию
torch.manual_seed(1337)


k = torch.randn(B, T, head_size)
q = torch.randn(B, T, head_size)
##-- without normalization
wei = q @ k.transpose(-2, -1)
print(f"k.var()={k.var():.2f}")
print(f"q.var()={q.var():.2f}")
print(f"wei.var()={wei.var():.2f}")

##-- with normalization
wei = q @ k.transpose(-2, -1) * head_size**-0.5
print(f"wei.var()={wei.var():.2f}")

k.var()=1.09
q.var()=1.06
wei.var()=18.44
wei.var()=1.15


In [39]:
##-- пример как ломается softmax
test = torch.tensor([0.1, -0.2, 0.3, -0.2, 0.5])
print(f"torch.softmax(test, dim=-1): {torch.softmax(test, dim=-1)}")
print(f"torch.softmax(test*8, dim=-1): {torch.softmax(test*8, dim=-1)}")
print(f"torch.softmax(test*80, dim=-1): {torch.softmax(test*80, dim=-1)}")
print(f"torch.softmax(test*800, dim=-1): {torch.softmax(test*800, dim=-1)}")

torch.softmax(test, dim=-1): tensor([0.1925, 0.1426, 0.2351, 0.1426, 0.2872])
torch.softmax(test*8, dim=-1): tensor([0.0326, 0.0030, 0.1615, 0.0030, 0.8000])
torch.softmax(test*80, dim=-1): tensor([1.2664e-14, 4.7809e-25, 1.1254e-07, 4.7809e-25, 1.0000e+00])
torch.softmax(test*800, dim=-1): tensor([0., 0., 0., 0., 1.])


In [40]:
wei[0]

tensor([[ 0.9710, -1.7965,  0.4228, -4.0060,  0.1322, -1.3341, -1.7085,  1.7138],
        [ 1.0812, -0.4803,  0.0384,  0.8505,  0.2169,  0.0100, -0.2031,  1.3135],
        [-0.5203,  0.8852,  0.2094, -0.4937, -0.0579, -0.1755,  0.0271, -0.7900],
        [-0.6459, -0.4687,  0.0900, -0.4316,  0.2010, -0.0377,  0.1445,  1.4498],
        [-0.4031,  0.1685, -1.0054, -2.8159, -1.1206, -0.4321,  0.6762, -1.1181],
        [-1.3248,  0.3587, -0.2602, -0.2964,  1.4291,  1.5470,  2.0224,  0.1044],
        [ 2.1717, -0.3078, -1.0187,  2.2616,  0.3535,  2.7194, -0.1230,  1.0794],
        [-0.7118,  0.5290,  1.8042,  1.1229,  1.1120, -1.1012, -0.3576,  0.5112]])

In [41]:
x[0]

tensor([[ 0.1808, -0.0700],
        [-0.3596, -0.9152],
        [ 0.6258,  0.0255],
        [ 0.9545,  0.0643],
        [ 0.3612,  1.1679],
        [-1.3499, -0.5102],
        [ 0.2360, -0.2398],
        [-0.9211,  1.5433]])

In [42]:
q[0]

tensor([[-0.6067,  1.8328,  0.2931,  0.0436,  2.1370, -0.6825, -1.6026, -0.1336,
          1.1407,  0.8935, -2.4000, -1.3420, -1.0023, -1.9870, -0.1217, -0.8074],
        [ 0.9255, -1.4814,  0.1291, -1.7058,  2.0330, -0.5062, -0.1016, -0.3016,
          1.2501, -0.2137, -1.1994, -0.8466, -0.3351,  1.0041,  0.8656,  0.1688],
        [-0.2352, -0.2586,  0.0131,  0.8719,  0.9102, -0.1875,  0.7229,  1.4742,
         -0.4048, -1.2273,  0.3382,  0.3641, -1.4508, -0.3814,  0.7220,  0.3461],
        [ 0.9967,  0.3575,  0.1187, -0.1062,  0.2990,  0.1199, -1.2433,  1.7859,
          0.9191,  0.2326,  0.4591,  0.2556, -0.3542,  0.6690,  0.7535, -0.5359],
        [-1.0277,  0.5347, -0.7958,  0.4780,  0.1588, -2.3106,  0.3227,  1.5431,
         -1.0392, -0.9678, -0.6961,  0.4684,  0.3586, -0.8773,  0.3267, -0.5667],
        [ 0.0438, -0.4455, -0.0267,  1.7926, -2.0707, -1.8788, -0.8275,  0.3157,
          1.2089,  0.2724,  0.0288,  0.9421, -1.5618,  1.0711,  0.4901, -0.4876],
        [ 1.2397, -2.1

In [43]:
k[0].T

tensor([[ 0.1808,  1.3488, -0.6631,  1.6455, -0.8345,  2.4900, -1.6669, -0.3296],
        [-0.0700, -0.1396, -0.2513, -0.8030,  0.5978, -1.2237, -1.3651,  0.0080],
        [-0.3596,  0.2858,  1.0101,  1.3514, -0.0514,  1.0107, -0.1655,  0.9262],
        [-0.9152,  0.9651,  0.1215, -0.2759, -0.0646,  0.5560,  0.9623, -1.8846],
        [ 0.6258, -2.0371,  0.1584, -1.5108, -0.4970, -1.5935,  0.0315,  0.1670],
        [ 0.0255,  0.4931,  1.1340,  2.1048,  0.4658, -1.2706, -0.7419,  0.4586],
        [ 0.9545,  1.4870, -1.1539,  2.7630, -0.2573,  0.6903, -0.2978, -1.7662],
        [ 0.0643,  0.5910, -0.2984, -1.7465, -1.0673, -0.1961,  0.0172,  0.5860],
        [ 0.3612,  0.1260, -0.5075,  1.4516,  2.0089,  0.3449, -0.1772,  1.7510],
        [ 1.1679, -1.5627, -0.9239, -1.5103, -0.5370, -0.3419, -0.1334,  0.2807],
        [-1.3499, -1.1601,  0.5467,  0.8212,  0.2228,  0.4759,  0.2940,  0.3110],
        [-0.5102, -0.3348, -1.4948, -0.2115,  0.6971, -0.7663,  1.3850, -0.6538],
        [ 0.2360

In [44]:
out[0]

tensor([[ 0.0660,  0.0865, -0.0022, -0.0979,  0.0494, -0.0847, -0.1617, -0.0495,
          0.1284,  0.1332,  0.0091,  0.0597,  0.1579, -0.0382,  0.0418, -0.0894],
        [-0.2420,  0.1175, -0.2201, -0.1949,  0.3228,  0.1415, -0.2377,  0.0728,
          0.0339,  0.2479,  0.1825, -0.0859,  0.2657, -0.0599, -0.1402, -0.2509],
        [ 0.0238,  0.1610, -0.0689, -0.2005,  0.1682, -0.0809, -0.3061, -0.0503,
          0.1969,  0.2677,  0.0678,  0.0612,  0.3090, -0.0734,  0.0190, -0.2049],
        [ 0.2719,  0.2343,  0.0551, -0.2476,  0.0619, -0.3013, -0.4325, -0.1734,
          0.3870,  0.3417, -0.0230,  0.2085,  0.4131, -0.1010,  0.1686, -0.2056],
        [ 0.1616,  0.1602,  0.0217, -0.1739,  0.0611, -0.1873, -0.2971, -0.1083,
          0.2543,  0.2386, -0.0032,  0.1304,  0.2863, -0.0697,  0.1009, -0.1501],
        [-0.6233, -0.2353, -0.2847,  0.1831,  0.2080,  0.5735,  0.4151,  0.3220,
         -0.5368, -0.2723,  0.2028, -0.3858, -0.3607,  0.0929, -0.3774,  0.0697],
        [ 0.1164,  0.0

In [45]:
#@title bigram_v3
bigram_v3 = \
"""
import torch
import torch.nn as nn
from torch.nn import functional as F

##-- hyperparameters
batch_size = 32 ##-- сколько независимых объектов (предложений) обрабатывается за раз (параллельно)
block_size = 8 ##-- максимальная длина предложения (в нашем случае, количество символов)
max_iters = 5000
eval_interval = 500
learning_rate = 1e-3
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200
n_embd = 32 ##-- размер эмбединга
##------------------

torch.manual_seed(1337)

# wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
with open('input.txt', 'r', encoding='utf-8') as file:
    text = file.read()

# here all the unique character that occur in this text
chars = sorted(list(set(text)))
vocab_size = len(chars)
##-- create mapping from characters to integers
stoi = { ch: i for i, ch in enumerate(chars) }
atoi = { i: ch for i, ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s]
decode = lambda l: ''.join([atoi[i] for i in l])

##-- Train and test splits
data = torch.tensor(encode(text), dtype=torch.long)
n = int(0.9 * len(data))
train_data = data[:n]
val_data = data[n:]

##-- data loading
def get_batch(split: str='train'):
    '''
    generate a small batch of data of inputs x and targets y

    split: str: train or test
    '''
    data = train_data if split=='train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size, )) ##-- random idx
    x = torch.stack([data[i: i+block_size] for i in ix])
    y = torch.stack([data[i+1: i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return (x, y)

@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out


class Head(nn.Module):
    ''' one head of self-attention '''

    def __init__(self, head_size):
        super().__init__()
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

    def forward(self, x):
        B, T, C = x.shape
        q = self.query(x) # (B, T, C)
        k = self.key(x) # (B, T, C)
        ##-- compute attention score 'affinities' (взаимоотношения между токенами)
        wei = q @ k.transpose(-2, -1) * C**-0.5 # (B, T, C) @ (B, C, T) -->> (B, T, T)
        wei = wei.masked_fill(self.tril[:T, :T]==0, float('-inf')) # (B, T, T)
        wei = F.softmax(wei, dim=-1)
        # perform the weighted aggregation of the value
        v = self.value(x) # (B, T, C)
        out = wei @ v  # (B, T, T) @ (B, T, C) -->> (B, T, C)
        return out


##-- super simple bigram model
class BigramLanguageModel(nn.Module):
    def __init__(self):
        super().__init__()
        ##-- each token directly read oof the logits for the next token from a lookip table
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd) ##-- берем эмбединг токена (кодируем токен)
        self.position_embedding_table = nn.Embedding(block_size, n_embd) ##-- кодируем позицию токена
        self.sa_head = Head(n_embd) ##-- one head of self-attention (B, T, C)
        self.lm_head = nn.Linear(n_embd, vocab_size) ##-- linear_model head

    def forward(self, idx, targets=None):
        B, T = idx.shape

        ##-- idx and targets are both (B, T) tensor of integers
        tok_emb = self.token_embedding_table(idx) # (B, T, C) #-- (4, 8, 32), (batch_size, block_size, emb_size)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device)) # (T, C) (8, 32) ##-- кодируем позицию токена
        x = tok_emb + pos_emb ##-- (B, T, C) эмбединг токена и его позиции
        x = self.sa_head(x) ##-- apple one head of self-attention (B, T, C)
        logits = self.lm_head(x) # (B, T, vocab_size) #-- (4, 8, 65), (batch_size, block_size, vocab_size)
        # print(logits.shape) # (4, 8, 65)
        # print(targets.shape) # (4, 8)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            ##-- из батча размером 4 предложения переводим в одну строку (то есть четыре предложения в одно)
            logits = logits.view(B*T, C)
            ##-- второй вариант, таргеты в таком случае тоже не менять
            # logits = torch.permute(logits, (0, 2, 1))
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
        return logits, loss

    def generate(self, idx, max_new_tokens: int=100):
        ##-- idx is the (B, T) array of the indices in the current context
        for _ in range(max_new_tokens):
            ##-- crop idx to the last block_size tokens
            idx_cond = idx[:, -block_size:]
            ##-- get the predictions
            logits, _ = self(idx_cond)
            ##-- focus only on the last time step (последнюю букву)
            # print(f"logits_before ={logits}, {logits.shape}")
            logits = logits[:, -1, :] # becomes (B, C)
            # print(f"logits_after ={logits}, {logits.shape}")
            ##-- softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # print(probs)
            ##-- sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # print(idx_next)
            ##-- append sampled idx to the running sample
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

model = BigramLanguageModel()
m = model.to(device)

##-- pytorch optimizer
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3)

##-- fit

for iter in range(max_iters):
    if iter % eval_interval == 0:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    ##-- sample a batch of data
    xb, yb = get_batch('train')

    ##-- evaluate the loss
    logits, loss = m(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

##-- generate the model
context = torch.zeros((1, 1), dtype=torch.long, device=device)
print(decode(m.generate(context, max_new_tokens=500)[0].tolist()))

"""


In [46]:
FILENAME = 'bigram_v3.py'
save(bigram_v3, FILENAME)

Saved file at: /content/bigram_v3.py


In [47]:
! diff bigram_v2.py bigram_v3.py

2d1
< 
8c7
< batch_size = 4 ##-- сколько независимых объектов (предложений) обрабатывается за раз (параллельно)
---
> batch_size = 32 ##-- сколько независимых объектов (предложений) обрабатывается за раз (параллельно)
10,12c9,11
< max_iters = 300
< eval_interval = 300
< learning_rate = 1e-2
---
> max_iters = 5000
> eval_interval = 500
> learning_rate = 1e-3
67a67,90
> class Head(nn.Module):
>     ''' one head of self-attention '''
> 
>     def __init__(self, head_size):
>         super().__init__()
>         self.query = nn.Linear(n_embd, head_size, bias=False)
>         self.key = nn.Linear(n_embd, head_size, bias=False)
>         self.value = nn.Linear(n_embd, head_size, bias=False)
>         self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))
> 
>     def forward(self, x):
>         B, T, C = x.shape
>         q = self.query(x) # (B, T, C)
>         k = self.key(x) # (B, T, C)
>         ##-- compute attention score 'affinities' (взаимоотношения между токенам

In [48]:
%%time

! python bigram_v3.py

step 0: train loss 4.1990, val loss 4.2031
step 500: train loss 2.6944, val loss 2.7091
step 1000: train loss 2.5214, val loss 2.5316
step 1500: train loss 2.4762, val loss 2.4833
step 2000: train loss 2.4408, val loss 2.4517
step 2500: train loss 2.4279, val loss 2.4433
step 3000: train loss 2.4126, val loss 2.4314
step 3500: train loss 2.3950, val loss 2.4190
step 4000: train loss 2.4033, val loss 2.3961
step 4500: train loss 2.3976, val loss 2.4041

Whent iknt,
Thowi, ht son, bth

Hiset bobe ale.
S:
O-' st dalilanss: rant he us he, vet?
Wed las ate awice my.

HDET:
ANGo oug
Yowns, tof isth bot mil ndill, aes iree sen cie lat Herid ovets, and Win ngarigoerabous lelind peal.
-hule onchiry ptugr aiss hew ye wllinde norod atelaves
Momy yowod mothake ont-wou whrt eiiby we ati fourive wee, ired thoouso er; thu the banterupt f so;
ARID Wam:
ENGCI inleront ffaf Pre?

Whiom.

He-
LIERCKENIGUICI
Sadsal aces ard thinin cour ay aney Iry ts I fr af ve y
CPU times: user 167 ms, sys: 33.5 ms, tota

# MultiHead attention

multihead attention - это просто параллельное выполнение нескольких видов attention и объединение (конкатенация) результатов.

Параллельный запуск нескольких голов помогает тем, что каждая голова собирает свою независимую информацию. Потом они объединяются и получают неплохой результат

In [49]:
#@title bigram_v4.py

bigram_v4 = \
"""
import torch
import torch.nn as nn
from torch.nn import functional as F

##-- hyperparameters
batch_size = 32 ##-- сколько независимых объектов (предложений) обрабатывается за раз (параллельно)
block_size = 8 ##-- максимальная длина предложения (в нашем случае, количество символов)
max_iters = 5000
eval_interval = 500
learning_rate = 1e-3
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200
n_embd = 32 ##-- размер эмбединга
##------------------

torch.manual_seed(1337)

# wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
with open('input.txt', 'r', encoding='utf-8') as file:
    text = file.read()

# here all the unique character that occur in this text
chars = sorted(list(set(text)))
vocab_size = len(chars)
##-- create mapping from characters to integers
stoi = { ch: i for i, ch in enumerate(chars) }
atoi = { i: ch for i, ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s]
decode = lambda l: ''.join([atoi[i] for i in l])

##-- Train and test splits
data = torch.tensor(encode(text), dtype=torch.long)
n = int(0.9 * len(data))
train_data = data[:n]
val_data = data[n:]

##-- data loading
def get_batch(split: str='train'):
    '''
    generate a small batch of data of inputs x and targets y

    split: str: train or test
    '''
    data = train_data if split=='train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size, )) ##-- random idx
    x = torch.stack([data[i: i+block_size] for i in ix])
    y = torch.stack([data[i+1: i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return (x, y)

@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out


class Head(nn.Module):
    ''' one head of self-attention '''

    def __init__(self, head_size):
        super().__init__()
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

    def forward(self, x):
        B, T, C = x.shape
        q = self.query(x) # (B, T, C)
        k = self.key(x) # (B, T, C)
        ##-- compute attention score 'affinities' (взаимоотношения между токенами)
        wei = q @ k.transpose(-2, -1) * C**-0.5 # (B, T, C) @ (B, C, T) -->> (B, T, T)
        wei = wei.masked_fill(self.tril[:T, :T]==0, float('-inf')) # (B, T, T)
        wei = F.softmax(wei, dim=-1)
        # perform the weighted aggregation of the value
        v = self.value(x) # (B, T, C)
        out = wei @ v  # (B, T, T) @ (B, T, C) -->> (B, T, C)
        return out


class MultiHeadAttention(nn.Module):
    ''' multiple heads of self-attention in parallel '''

    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])

    def forward(self, x):
        return torch.cat([h(x) for h in self.heads], dim=-1) ##-- объединение по каналам


##-- super simple bigram model
class BigramLanguageModel(nn.Module):
    def __init__(self):
        super().__init__()
        ##-- each token directly read oof the logits for the next token from a lookip table
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd) ##-- берем эмбединг токена (кодируем токен)
        self.position_embedding_table = nn.Embedding(block_size, n_embd) ##-- кодируем позицию токена
        self.sa_heads = MultiHeadAttention(4, n_embd // 4) ##-- (B, T, C) 4 головы, каждая размером 8 (то есть на выходе эмбединг = 32)
        self.lm_head = nn.Linear(n_embd, vocab_size) ##-- linear_model head

    def forward(self, idx, targets=None):
        B, T = idx.shape

        ##-- idx and targets are both (B, T) tensor of integers
        tok_emb = self.token_embedding_table(idx) # (B, T, C) #-- (4, 8, 32), (batch_size, block_size, emb_size)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device)) # (T, C) (8, 32) ##-- кодируем позицию токена
        x = tok_emb + pos_emb ##-- (B, T, C) эмбединг токена и его позиции
        x = self.sa_heads(x) ##-- apple one head of self-attention (B, T, C)
        logits = self.lm_head(x) # (B, T, vocab_size) #-- (4, 8, 65), (batch_size, block_size, vocab_size)
        # print(logits.shape) # (4, 8, 65)
        # print(targets.shape) # (4, 8)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            ##-- из батча размером 4 предложения переводим в одну строку (то есть четыре предложения в одно)
            logits = logits.view(B*T, C)
            ##-- второй вариант, таргеты в таком случае тоже не менять
            # logits = torch.permute(logits, (0, 2, 1))
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
        return logits, loss

    def generate(self, idx, max_new_tokens: int=100):
        ##-- idx is the (B, T) array of the indices in the current context
        for _ in range(max_new_tokens):
            ##-- crop idx to the last block_size tokens
            idx_cond = idx[:, -block_size:]
            ##-- get the predictions
            logits, _ = self(idx_cond)
            ##-- focus only on the last time step (последнюю букву)
            # print(f"logits_before ={logits}, {logits.shape}")
            logits = logits[:, -1, :] # becomes (B, C)
            # print(f"logits_after ={logits}, {logits.shape}")
            ##-- softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # print(probs)
            ##-- sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # print(idx_next)
            ##-- append sampled idx to the running sample
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

model = BigramLanguageModel()
m = model.to(device)

##-- pytorch optimizer
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3)

##-- fit

for iter in range(max_iters):
    if iter % eval_interval == 0:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    ##-- sample a batch of data
    xb, yb = get_batch('train')

    ##-- evaluate the loss
    logits, loss = m(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

##-- generate the model
context = torch.zeros((1, 1), dtype=torch.long, device=device)
print(decode(m.generate(context, max_new_tokens=500)[0].tolist()))

"""

In [50]:
FILENAME = 'bigram_v4.py'
save(bigram_v4, FILENAME)

Saved file at: /content/bigram_v4.py


In [51]:
! diff bigram_v3.py bigram_v4.py

90a91,101
> class MultiHeadAttention(nn.Module):
>     ''' multiple heads of self-attention in parallel '''
> 
>     def __init__(self, num_heads, head_size):
>         super().__init__()
>         self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
> 
>     def forward(self, x):
>         return torch.cat([h(x) for h in self.heads], dim=-1) ##-- объединение по каналам
> 
> 
98c109
<         self.sa_head = Head(n_embd) ##-- one head of self-attention (B, T, C)
---
>         self.sa_heads = MultiHeadAttention(4, n_embd // 4) ##-- (B, T, C) 4 головы, каждая размером 8 (то есть на выходе эмбединг = 32)
108c119
<         x = self.sa_head(x) ##-- apple one head of self-attention (B, T, C)
---
>         x = self.sa_heads(x) ##-- apple one head of self-attention (B, T, C)


In [52]:
%%time

! python bigram_v4.py

step 0: train loss 4.2229, val loss 4.2220
step 500: train loss 2.6454, val loss 2.6623
step 1000: train loss 2.4841, val loss 2.4949
step 1500: train loss 2.4184, val loss 2.4251
step 2000: train loss 2.3641, val loss 2.3774
step 2500: train loss 2.3359, val loss 2.3473
step 3000: train loss 2.3119, val loss 2.3272
step 3500: train loss 2.2861, val loss 2.3109
step 4000: train loss 2.2852, val loss 2.2783
step 4500: train loss 2.2679, val loss 2.2789

Whent if tridcowd, whis by bee

Hiret bobe toe.
Sagrtand wealleauss:
Waith fuulqure vet?
W dilth ate arcce my.

ODET:
An oroug
Yowns, tof is heice mil; dill, aeg isee sen cin lat Helilrov the and Wing.

Don you sesel lind peall lish wonchiry prug; aiss hiw yevellinge nordoperelaves
Mom
Is wod mothake onWinso whroCeriby wey thourstis The shiend to-o--
And; the the banterthirf son; igr to whit thy ale of,
Toff Prive my of.

HINNGIES:
Wied is:
Sadsal aces ghe thiuin couk ay andy Iur to I frouf vouc
CPU times: user 256 ms, sys: 50.5 ms, tota

# Feed Forward Network

Это можно представить как "обдумывание".

Multihead attention позволяла токенам "пообщаться" друг с другом, то есть узнать друг друга. А feedforward как бы дает паузу, чтобы обдумать то, что они узнали друг о друге.

In [53]:
#@title bigram_v5
bigram_v5 = \
"""
import torch
import torch.nn as nn
from torch.nn import functional as F

##-- hyperparameters
batch_size = 32 ##-- сколько независимых объектов (предложений) обрабатывается за раз (параллельно)
block_size = 8 ##-- максимальная длина предложения (в нашем случае, количество символов)
max_iters = 5000
eval_interval = 500
learning_rate = 1e-3
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200
n_embd = 32 ##-- размер эмбединга
##------------------

torch.manual_seed(1337)

# wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
with open('input.txt', 'r', encoding='utf-8') as file:
    text = file.read()

# here all the unique character that occur in this text
chars = sorted(list(set(text)))
vocab_size = len(chars)
##-- create mapping from characters to integers
stoi = { ch: i for i, ch in enumerate(chars) }
atoi = { i: ch for i, ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s]
decode = lambda l: ''.join([atoi[i] for i in l])

##-- Train and test splits
data = torch.tensor(encode(text), dtype=torch.long)
n = int(0.9 * len(data))
train_data = data[:n]
val_data = data[n:]

##-- data loading
def get_batch(split: str='train'):
    '''
    generate a small batch of data of inputs x and targets y

    split: str: train or test
    '''
    data = train_data if split=='train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size, )) ##-- random idx
    x = torch.stack([data[i: i+block_size] for i in ix])
    y = torch.stack([data[i+1: i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return (x, y)

@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out


class Head(nn.Module):
    ''' one head of self-attention '''

    def __init__(self, head_size):
        super().__init__()
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

    def forward(self, x):
        B, T, C = x.shape
        q = self.query(x) # (B, T, C)
        k = self.key(x) # (B, T, C)
        ##-- compute attention score 'affinities' (взаимоотношения между токенами)
        wei = q @ k.transpose(-2, -1) * C**-0.5 # (B, T, C) @ (B, C, T) -->> (B, T, T)
        wei = wei.masked_fill(self.tril[:T, :T]==0, float('-inf')) # (B, T, T)
        wei = F.softmax(wei, dim=-1)
        # perform the weighted aggregation of the value
        v = self.value(x) # (B, T, C)
        out = wei @ v  # (B, T, T) @ (B, T, C) -->> (B, T, C)
        return out


class MultiHeadAttention(nn.Module):
    ''' multiple heads of self-attention in parallel '''

    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])

    def forward(self, x):
        return torch.cat([h(x) for h in self.heads], dim=-1) ##-- объединение по каналам


class FeedForward(nn.Module):
    ''' a simple linear layer followed by a non-linearity '''

    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, n_embd),
            nn.ReLU()
        )

    def forward(self, x):
        return self.net(x)


##-- super simple bigram model
class BigramLanguageModel(nn.Module):
    def __init__(self):
        super().__init__()
        ##-- each token directly read oof the logits for the next token from a lookip table
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd) ##-- берем эмбединг токена (кодируем токен)
        self.position_embedding_table = nn.Embedding(block_size, n_embd) ##-- кодируем позицию токена
        self.sa_heads = MultiHeadAttention(4, n_embd // 4) ##-- (B, T, C) 4 головы, каждая размером 8 (то есть на выходе эмбединг = 32)
        self.ffwd = FeedForward(n_embd)
        self.lm_head = nn.Linear(n_embd, vocab_size) ##-- linear_model head

    def forward(self, idx, targets=None):
        B, T = idx.shape

        ##-- idx and targets are both (B, T) tensor of integers
        tok_emb = self.token_embedding_table(idx) # (B, T, C) #-- (4, 8, 32), (batch_size, block_size, emb_size)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device)) # (T, C) (8, 32) ##-- кодируем позицию токена
        x = tok_emb + pos_emb ##-- (B, T, C) эмбединг токена и его позиции
        x = self.sa_heads(x) ##-- apply 4 heads of self-attention (B, T, C)
        x = self.ffwd(x) ##-- (B, T, C)
        logits = self.lm_head(x) # (B, T, vocab_size) #-- (4, 8, 65), (batch_size, block_size, vocab_size)
        # print(logits.shape) # (4, 8, 65)
        # print(targets.shape) # (4, 8)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            ##-- из батча размером 4 предложения переводим в одну строку (то есть четыре предложения в одно)
            logits = logits.view(B*T, C)
            ##-- второй вариант, таргеты в таком случае тоже не менять
            # logits = torch.permute(logits, (0, 2, 1))
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
        return logits, loss

    def generate(self, idx, max_new_tokens: int=100):
        ##-- idx is the (B, T) array of the indices in the current context
        for _ in range(max_new_tokens):
            ##-- crop idx to the last block_size tokens
            idx_cond = idx[:, -block_size:]
            ##-- get the predictions
            logits, _ = self(idx_cond)
            ##-- focus only on the last time step (последнюю букву)
            # print(f"logits_before ={logits}, {logits.shape}")
            logits = logits[:, -1, :] # becomes (B, C)
            # print(f"logits_after ={logits}, {logits.shape}")
            ##-- softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # print(probs)
            ##-- sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # print(idx_next)
            ##-- append sampled idx to the running sample
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

model = BigramLanguageModel()
m = model.to(device)

##-- pytorch optimizer
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3)

##-- fit

for iter in range(max_iters):
    if iter % eval_interval == 0:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    ##-- sample a batch of data
    xb, yb = get_batch('train')

    ##-- evaluate the loss
    logits, loss = m(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

##-- generate the model
context = torch.zeros((1, 1), dtype=torch.long, device=device)
print(decode(m.generate(context, max_new_tokens=500)[0].tolist()))

"""

In [54]:
FILENAME = 'bigram_v5.py'
save(bigram_v5, FILENAME)

Saved file at: /content/bigram_v5.py


In [55]:
! diff bigram_v4.py bigram_v5.py

101a102,115
> class FeedForward(nn.Module):
>     ''' a simple linear layer followed by a non-linearity '''
> 
>     def __init__(self, n_embd):
>         super().__init__()
>         self.net = nn.Sequential(
>             nn.Linear(n_embd, n_embd),
>             nn.ReLU()
>         )
> 
>     def forward(self, x):
>         return self.net(x)
> 
> 
109a124
>         self.ffwd = FeedForward(n_embd)
119c134,135
<         x = self.sa_heads(x) ##-- apple one head of self-attention (B, T, C)
---
>         x = self.sa_heads(x) ##-- apply 4 heads of self-attention (B, T, C)
>         x = self.ffwd(x) ##-- (B, T, C)


In [56]:
%%time

! python bigram_v5.py

step 0: train loss 4.1994, val loss 4.1993
step 500: train loss 2.5829, val loss 2.5937
step 1000: train loss 2.4608, val loss 2.4632
step 1500: train loss 2.3987, val loss 2.3980
step 2000: train loss 2.3334, val loss 2.3507
step 2500: train loss 2.3051, val loss 2.3284
step 3000: train loss 2.2896, val loss 2.3052
step 3500: train loss 2.2568, val loss 2.2849
step 4000: train loss 2.2486, val loss 2.2559
step 4500: train loss 2.2336, val loss 2.2478

Will thy tridcow and is and the a thom obe to tavegr-and thalllands:
Waith foulquethe thar dill
Haten wice my.

Hnd acut onoth
Yowthertof is he cove the ill, aed iree sen cin lat Hetilkev the and to por:
Wilerans!
 lelind teall thull cechir sother aiss hiwty.
Hur nie norfore tigtle
Mom
Gll, demethake on indo whre piiby, to thour rive ceeshime st so mowe-xore
To k danteruft for tre
QULETE:

Enge inled ath, af Pried my ome.
He-Nuck!

Wind is:
Sadsal the E'd st hoth cour aar tey Ire to cof my cardy
CPU times: user 274 ms, sys: 52.5 ms, tota

# Block (Multihead attention + Feedforward several times)

Добавление только blocks только ухудшает качество модели, так как модель становится глубокой, то происходит затухание градиентов. Чтобы это исправить, нужно добавить residual connection



In [57]:
#@title bigram_v6
bigram_v6 = \
"""

import torch
import torch.nn as nn
from torch.nn import functional as F

##-- hyperparameters
batch_size = 32 ##-- сколько независимых объектов (предложений) обрабатывается за раз (параллельно)
block_size = 8 ##-- максимальная длина предложения (в нашем случае, количество символов)
max_iters = 5000
eval_interval = 500
learning_rate = 1e-3
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200
n_embd = 32 ##-- размер эмбединга
##------------------

torch.manual_seed(1337)

# wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
with open('input.txt', 'r', encoding='utf-8') as file:
    text = file.read()

# here all the unique character that occur in this text
chars = sorted(list(set(text)))
vocab_size = len(chars)
##-- create mapping from characters to integers
stoi = { ch: i for i, ch in enumerate(chars) }
atoi = { i: ch for i, ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s]
decode = lambda l: ''.join([atoi[i] for i in l])

##-- Train and test splits
data = torch.tensor(encode(text), dtype=torch.long)
n = int(0.9 * len(data))
train_data = data[:n]
val_data = data[n:]

##-- data loading
def get_batch(split: str='train'):
    '''
    generate a small batch of data of inputs x and targets y

    split: str: train or test
    '''
    data = train_data if split=='train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size, )) ##-- random idx
    x = torch.stack([data[i: i+block_size] for i in ix])
    y = torch.stack([data[i+1: i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return (x, y)

@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out


class Head(nn.Module):
    ''' one head of self-attention '''

    def __init__(self, head_size):
        super().__init__()
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

    def forward(self, x):
        B, T, C = x.shape
        q = self.query(x) # (B, T, C)
        k = self.key(x) # (B, T, C)
        ##-- compute attention score 'affinities' (взаимоотношения между токенами)
        wei = q @ k.transpose(-2, -1) * C**-0.5 # (B, T, C) @ (B, C, T) -->> (B, T, T)
        wei = wei.masked_fill(self.tril[:T, :T]==0, float('-inf')) # (B, T, T)
        wei = F.softmax(wei, dim=-1)
        # perform the weighted aggregation of the value
        v = self.value(x) # (B, T, C)
        out = wei @ v  # (B, T, T) @ (B, T, C) -->> (B, T, C)
        return out


class MultiHeadAttention(nn.Module):
    ''' multiple heads of self-attention in parallel '''

    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])

    def forward(self, x):
        return torch.cat([h(x) for h in self.heads], dim=-1) ##-- объединение по каналам


class FeedForward(nn.Module):
    ''' a simple linear layer followed by a non-linearity '''

    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, n_embd),
            nn.ReLU()
        )

    def forward(self, x):
        return self.net(x)


class Block(nn.Module):
    ''' Transformer block: communication followed by computation '''

    def __init__(self, n_embd, n_head):
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedForward(n_embd)
    
    def forward(self, x):
        x = self.sa(x)
        x = self.ffwd(x)
        return x


##-- super simple bigram model
class BigramLanguageModel(nn.Module):
    def __init__(self):
        super().__init__()
        ##-- each token directly read oof the logits for the next token from a lookip table
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd) ##-- берем эмбединг токена (кодируем токен)
        self.position_embedding_table = nn.Embedding(block_size, n_embd) ##-- кодируем позицию токена
        self.blocks = nn.Sequential(
            Block(n_embd, n_head=4),
            Block(n_embd, n_head=4),
            Block(n_embd, n_head=4),
        ) ##-- MultiheadAttention -> FFN 3 times
        self.lm_head = nn.Linear(n_embd, vocab_size) ##-- linear_model head

    def forward(self, idx, targets=None):
        B, T = idx.shape

        ##-- idx and targets are both (B, T) tensor of integers
        tok_emb = self.token_embedding_table(idx) # (B, T, C) #-- (4, 8, 32), (batch_size, block_size, emb_size)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device)) # (T, C) (8, 32) ##-- кодируем позицию токена
        x = tok_emb + pos_emb ##-- (B, T, C) эмбединг токена и его позиции
        x = self.blocks(x) ##-- apply 4 heads of self-attention (B, T, C)
        logits = self.lm_head(x) # (B, T, vocab_size) #-- (4, 8, 65), (batch_size, block_size, vocab_size)
        # print(logits.shape) # (4, 8, 65)
        # print(targets.shape) # (4, 8)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            ##-- из батча размером 4 предложения переводим в одну строку (то есть четыре предложения в одно)
            logits = logits.view(B*T, C)
            ##-- второй вариант, таргеты в таком случае тоже не менять
            # logits = torch.permute(logits, (0, 2, 1))
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
        return logits, loss

    def generate(self, idx, max_new_tokens: int=100):
        ##-- idx is the (B, T) array of the indices in the current context
        for _ in range(max_new_tokens):
            ##-- crop idx to the last block_size tokens
            idx_cond = idx[:, -block_size:]
            ##-- get the predictions
            logits, _ = self(idx_cond)
            ##-- focus only on the last time step (последнюю букву)
            # print(f"logits_before ={logits}, {logits.shape}")
            logits = logits[:, -1, :] # becomes (B, C)
            # print(f"logits_after ={logits}, {logits.shape}")
            ##-- softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # print(probs)
            ##-- sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # print(idx_next)
            ##-- append sampled idx to the running sample
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

model = BigramLanguageModel()
m = model.to(device)

##-- pytorch optimizer
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3)

##-- fit

for iter in range(max_iters):
    if iter % eval_interval == 0:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    ##-- sample a batch of data
    xb, yb = get_batch('train')

    ##-- evaluate the loss
    logits, loss = m(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

##-- generate the model
context = torch.zeros((1, 1), dtype=torch.long, device=device)
print(decode(m.generate(context, max_new_tokens=500)[0].tolist()))

"""

In [58]:
FILENAME = 'bigram_v6.py'
save(bigram_v6, FILENAME)

Saved file at: /content/bigram_v6.py


In [59]:
! diff bigram_v5.py bigram_v6.py

1a2
> 
115a117,131
> class Block(nn.Module):
>     ''' Transformer block: communication followed by computation '''
> 
>     def __init__(self, n_embd, n_head):
>         super().__init__()
>         head_size = n_embd // n_head
>         self.sa = MultiHeadAttention(n_head, head_size)
>         self.ffwd = FeedForward(n_embd)
>     
>     def forward(self, x):
>         x = self.sa(x)
>         x = self.ffwd(x)
>         return x
> 
> 
123,124c139,143
<         self.sa_heads = MultiHeadAttention(4, n_embd // 4) ##-- (B, T, C) 4 головы, каждая размером 8 (то есть на выходе эмбединг = 32)
<         self.ffwd = FeedForward(n_embd)
---
>         self.blocks = nn.Sequential(
>             Block(n_embd, n_head=4),
>             Block(n_embd, n_head=4),
>             Block(n_embd, n_head=4),
>         ) ##-- MultiheadAttention -> FFN 3 times
134,135c153
<         x = self.sa_heads(x) ##-- apply 4 heads of self-attention (B, T, C)
<         x = self.ffwd(x) ##-- (B, T, C)
---
>         x = se

In [60]:
%%time

! python bigram_v6.py

step 0: train loss 4.2116, val loss 4.2078
step 500: train loss 3.1064, val loss 3.1091
step 1000: train loss 2.8667, val loss 2.8511
step 1500: train loss 2.6722, val loss 2.6536
step 2000: train loss 2.5680, val loss 2.5777
step 2500: train loss 2.5047, val loss 2.4935
step 3000: train loss 2.4558, val loss 2.4622
step 3500: train loss 2.4198, val loss 2.4349
step 4000: train loss 2.4070, val loss 2.3855
step 4500: train loss 2.3726, val loss 2.3828

And thef bridc wilen is by bee
baceer bobe dontaSthr-' my dalata
he ar hapae uwqore to barddt
ASoates wice my thans'n on onom waoths
Mtof is heing mil nowl boues iree sen cin lat Het drovets, and to pomant.

Wabs!
Al lind me litens;
Honc my sothpe anss hew younls mas korfd atelg dowhom
thaked mothakleo Wind to do eiibas to thours
Wor cens ired to-Lcol erxore
To kad nonruft fis to irk to she wrey aleront to fim.

CORISos.

HKINLIES:
Toy dis:
Sadsal thes gef thiuin couk aar tey Ire to chan you! hi
CPU times: user 530 ms, sys: 107 ms, total

# Residual connection

Residual connection (соединение остаточной связи) - это механизм, который используется в нейронных сетях для передачи информации из одного слоя в другой, обеспечивая более эффективное обучение. Он заключается в том, что на каждом слое данных добавляется остаточное соединение, которое передает информацию от входа к выходу слоя, минуя блоки обработки данных. Это позволяет улучшить качество обучения и снизить вероятность переобучения модели.

residual connection сохраняет и передает информацию, которая может быть полезной для более поздних этапов нейронной сети. Это позволяет улучшить качество предсказаний и ускорить обучение сети.

In [61]:
#@title bigram_v7
bigram_v7 = \
"""


import torch
import torch.nn as nn
from torch.nn import functional as F

##-- hyperparameters
batch_size = 32 ##-- сколько независимых объектов (предложений) обрабатывается за раз (параллельно)
block_size = 8 ##-- максимальная длина предложения (в нашем случае, количество символов)
max_iters = 5000
eval_interval = 500
learning_rate = 1e-3
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200
n_embd = 32 ##-- размер эмбединга
##------------------

torch.manual_seed(1337)

# wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
with open('input.txt', 'r', encoding='utf-8') as file:
    text = file.read()

# here all the unique character that occur in this text
chars = sorted(list(set(text)))
vocab_size = len(chars)
##-- create mapping from characters to integers
stoi = { ch: i for i, ch in enumerate(chars) }
atoi = { i: ch for i, ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s]
decode = lambda l: ''.join([atoi[i] for i in l])

##-- Train and test splits
data = torch.tensor(encode(text), dtype=torch.long)
n = int(0.9 * len(data))
train_data = data[:n]
val_data = data[n:]

##-- data loading
def get_batch(split: str='train'):
    '''
    generate a small batch of data of inputs x and targets y

    split: str: train or test
    '''
    data = train_data if split=='train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size, )) ##-- random idx
    x = torch.stack([data[i: i+block_size] for i in ix])
    y = torch.stack([data[i+1: i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return (x, y)

@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out


class Head(nn.Module):
    ''' one head of self-attention '''

    def __init__(self, head_size):
        super().__init__()
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

    def forward(self, x):
        B, T, C = x.shape
        q = self.query(x) # (B, T, C)
        k = self.key(x) # (B, T, C)
        ##-- compute attention score 'affinities' (взаимоотношения между токенами)
        wei = q @ k.transpose(-2, -1) * C**-0.5 # (B, T, C) @ (B, C, T) -->> (B, T, T)
        wei = wei.masked_fill(self.tril[:T, :T]==0, float('-inf')) # (B, T, T)
        wei = F.softmax(wei, dim=-1)
        # perform the weighted aggregation of the value
        v = self.value(x) # (B, T, C)
        out = wei @ v  # (B, T, T) @ (B, T, C) -->> (B, T, C)
        return out


class MultiHeadAttention(nn.Module):
    ''' multiple heads of self-attention in parallel '''

    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(n_embd, n_embd) ##-- for residual connection

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1) ##-- объединение по каналам
        out = self.proj(out)
        return out


class FeedForward(nn.Module):
    ''' a simple linear layer followed by a non-linearity '''

    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd) ##-- for residual connection
        )

    def forward(self, x):
        return self.net(x)


class Block(nn.Module):
    ''' Transformer block: communication followed by computation '''

    def __init__(self, n_embd, n_head):
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedForward(n_embd)
    
    def forward(self, x):
        x = x + self.sa(x) ##-- add residual connection
        x = x + self.ffwd(x) ##-- add residual connection
        return x


##-- super simple bigram model
class BigramLanguageModel(nn.Module):
    def __init__(self):
        super().__init__()
        ##-- each token directly read oof the logits for the next token from a lookip table
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd) ##-- берем эмбединг токена (кодируем токен)
        self.position_embedding_table = nn.Embedding(block_size, n_embd) ##-- кодируем позицию токена
        self.blocks = nn.Sequential(
            Block(n_embd, n_head=4),
            Block(n_embd, n_head=4),
            Block(n_embd, n_head=4),
        ) ##-- MultiheadAttention -> FFN 3 times
        self.lm_head = nn.Linear(n_embd, vocab_size) ##-- linear_model head

    def forward(self, idx, targets=None):
        B, T = idx.shape

        ##-- idx and targets are both (B, T) tensor of integers
        tok_emb = self.token_embedding_table(idx) # (B, T, C) #-- (4, 8, 32), (batch_size, block_size, emb_size)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device)) # (T, C) (8, 32) ##-- кодируем позицию токена
        x = tok_emb + pos_emb ##-- (B, T, C) эмбединг токена и его позиции
        x = self.blocks(x) ##-- apply 4 heads of self-attention (B, T, C)
        logits = self.lm_head(x) # (B, T, vocab_size) #-- (4, 8, 65), (batch_size, block_size, vocab_size)
        # print(logits.shape) # (4, 8, 65)
        # print(targets.shape) # (4, 8)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            ##-- из батча размером 4 предложения переводим в одну строку (то есть четыре предложения в одно)
            logits = logits.view(B*T, C)
            ##-- второй вариант, таргеты в таком случае тоже не менять
            # logits = torch.permute(logits, (0, 2, 1))
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
        return logits, loss

    def generate(self, idx, max_new_tokens: int=100):
        ##-- idx is the (B, T) array of the indices in the current context
        for _ in range(max_new_tokens):
            ##-- crop idx to the last block_size tokens
            idx_cond = idx[:, -block_size:]
            ##-- get the predictions
            logits, _ = self(idx_cond)
            ##-- focus only on the last time step (последнюю букву)
            # print(f"logits_before ={logits}, {logits.shape}")
            logits = logits[:, -1, :] # becomes (B, C)
            # print(f"logits_after ={logits}, {logits.shape}")
            ##-- softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # print(probs)
            ##-- sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # print(idx_next)
            ##-- append sampled idx to the running sample
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

model = BigramLanguageModel()
m = model.to(device)

##-- pytorch optimizer
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3)

##-- fit

for iter in range(max_iters):
    if iter % eval_interval == 0:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    ##-- sample a batch of data
    xb, yb = get_batch('train')

    ##-- evaluate the loss
    logits, loss = m(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

##-- generate the model
context = torch.zeros((1, 1), dtype=torch.long, device=device)
print(decode(m.generate(context, max_new_tokens=500)[0].tolist()))

"""

In [62]:
FILENAME = 'bigram_v7.py'
save(bigram_v7, FILENAME)

Saved file at: /content/bigram_v7.py


In [63]:
! diff bigram_v6.py bigram_v7.py

2a3
> 
97a99
>         self.proj = nn.Linear(n_embd, n_embd) ##-- for residual connection
100c102,104
<         return torch.cat([h(x) for h in self.heads], dim=-1) ##-- объединение по каналам
---
>         out = torch.cat([h(x) for h in self.heads], dim=-1) ##-- объединение по каналам
>         out = self.proj(out)
>         return out
109,110c113,115
<             nn.Linear(n_embd, n_embd),
<             nn.ReLU()
---
>             nn.Linear(n_embd, 4 * n_embd),
>             nn.ReLU(),
>             nn.Linear(4 * n_embd, n_embd) ##-- for residual connection
127,128c132,133
<         x = self.sa(x)
<         x = self.ffwd(x)
---
>         x = x + self.sa(x) ##-- add residual connection
>         x = x + self.ffwd(x) ##-- add residual connection


In [64]:
%%time

! python bigram_v7.py

step 0: train loss 4.6208, val loss 4.6182
step 500: train loss 2.3855, val loss 2.3836
step 1000: train loss 2.2681, val loss 2.2638
step 1500: train loss 2.1911, val loss 2.2064
step 2000: train loss 2.1506, val loss 2.1866
step 2500: train loss 2.1001, val loss 2.1436
step 3000: train loss 2.0745, val loss 2.1436
step 3500: train loss 2.0583, val loss 2.1167
step 4000: train loss 2.0274, val loss 2.1081
step 4500: train loss 2.0051, val loss 2.1078

And they bridcewill, is by be madise, bube a enave Our my dalled
here'd this us comen. Warderlascanessway, my fearstard, onour
You, proof is hear this now
Whime misters, hand latist in ov the and the now on you meselvinnce, all thus would by prup; and all, ye will, she rookes, a down'd
the the to kelloal---on her eviby, with to must and hipence poocter,-for und kind thrupt for ar igner must with ale of wafife
Rere my of.

HENNS:
Hast you as
ardsal the age stavein cour aar tey it his chan you!
An
CPU times: user 625 ms, sys: 99.5 ms, tota

# LayerNorm

LayerNorm (Layer Normalization) - это метод нормализации данных, который применяется в нейронных сетях для ускорения обучения и повышения стабильности сходимости.

В отличие от Batch Normalization, который выполняет нормализацию по батчу, LayerNorm нормализует данные по каждому нейрону в слое. Таким образом, LayerNorm не зависит от размера батча и может быть использован для обработки данных в режиме онлайн.

Формально, LayerNorm применяет нормализацию к входным данным $x$ по среднему значению $\mu$ и стандартному отклонению $\sigma$ по каждому признаку (нейрону) внутри слоя:


$\text{LayerNorm}(x_i) = \frac{x_i - \mu}{\sqrt{\sigma^2 + \epsilon}} \odot \gamma + \beta$

где $\epsilon$ - это малое число, добавленное для стабильности вычислений, $\gamma$ и $\beta$ - это обучаемые параметры, которые могут быть использованы для регулирования масштаба и смещения данных после нормализации.

Применение LayerNorm позволяет уменьшить внутреннюю зависимость между параметрами и ускорить сходимость при обучении нейронных сетей.

In [65]:
class LayerNorm:

    def __init__(self, dim, eps=1e-5, momentum=0.1):
        self.eps = eps
        self.gamma = torch.ones(dim)
        self.beta = torch.zeros(dim)

    def __call__(self, x):
        ##-- calculate the forward pass
        xmean = x.mean(1, keepdim=True) ##-- mean by row
        xvar = x.var(1, keepdim=True)
        xhat = (x - xmean) / torch.sqrt(xvar + self.eps) ##-- normalize
        self.out = self.gamma * xhat + self.beta
        return self.out

    def parameters(self):
        return [self.gamma, self.beta]

torch.manual_seed(1337)
module = LayerNorm(100)
x = torch.randn(32, 100)
x = module(x)
x.shape

torch.Size([32, 100])

In [66]:
x.mean(1), x.var(1) ##-- по строке среднее равно нулю и СКО = 0

(tensor([-9.5367e-09, -2.3842e-09, -2.0266e-08,  1.7881e-08,  1.6689e-08,
          9.8348e-09,  4.7684e-09,  1.9073e-08, -1.4305e-08, -4.7684e-09,
         -1.3113e-08, -5.9605e-09,  0.0000e+00, -7.1526e-09, -2.0266e-08,
          7.0035e-09, -1.2815e-08,  1.7881e-08,  6.5565e-09, -4.7684e-09,
          9.5367e-09, -3.5763e-09, -2.8610e-08,  4.7684e-09,  3.5763e-09,
         -7.1526e-09, -4.7684e-09,  0.0000e+00,  5.3644e-09, -1.1921e-08,
          4.7684e-09,  1.9073e-08]),
 tensor([1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
         1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
         1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
         1.0000, 1.0000, 1.0000, 1.0000, 1.0000]))

In [67]:
x.mean(0), x.var(0) ##-- по столбцам - нет

(tensor([ 0.1469, -0.5910, -0.3974,  0.0468, -0.1431,  0.0138, -0.2664,  0.4181,
          0.1426,  0.2191,  0.2554, -0.2625, -0.0543, -0.1050,  0.1541,  0.2492,
          0.2498,  0.1354, -0.2027, -0.3772,  0.2920,  0.1959, -0.2249, -0.0574,
          0.1293, -0.1413,  0.1445, -0.2509,  0.1434,  0.0128,  0.0631, -0.2482,
         -0.0977,  0.0945,  0.1880,  0.0951,  0.0047,  0.2833,  0.1154, -0.3063,
          0.0510,  0.1602,  0.0598,  0.1157,  0.0083, -0.2541, -0.0447, -0.0921,
          0.1891, -0.0150, -0.1857, -0.4513, -0.1106,  0.0320,  0.0417,  0.1272,
         -0.3022, -0.2864,  0.2507, -0.1101,  0.0402,  0.2277,  0.2753,  0.2577,
         -0.1698,  0.2775, -0.1854,  0.0767, -0.2023,  0.2106,  0.1443,  0.1391,
          0.1628,  0.1442, -0.0223, -0.0108,  0.0173,  0.0508, -0.0126,  0.1257,
          0.0073,  0.0803, -0.0412,  0.0335, -0.2576, -0.0642, -0.1877, -0.1026,
         -0.0888,  0.1562,  0.1422,  0.0295,  0.1925, -0.2169, -0.1416,  0.1693,
         -0.1704, -0.2708,  

In [68]:
#@title bigram_v8
bigram_v8 = \
"""
import torch
import torch.nn as nn
from torch.nn import functional as F

##-- hyperparameters
batch_size = 32 ##-- сколько независимых объектов (предложений) обрабатывается за раз (параллельно)
block_size = 8 ##-- максимальная длина предложения (в нашем случае, количество символов)
max_iters = 5000
eval_interval = 500
learning_rate = 1e-3
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200
n_embd = 32 ##-- размер эмбединга
##------------------

torch.manual_seed(1337)

# wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
with open('input.txt', 'r', encoding='utf-8') as file:
    text = file.read()

# here all the unique character that occur in this text
chars = sorted(list(set(text)))
vocab_size = len(chars)
##-- create mapping from characters to integers
stoi = { ch: i for i, ch in enumerate(chars) }
atoi = { i: ch for i, ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s]
decode = lambda l: ''.join([atoi[i] for i in l])

##-- Train and test splits
data = torch.tensor(encode(text), dtype=torch.long)
n = int(0.9 * len(data))
train_data = data[:n]
val_data = data[n:]

##-- data loading
def get_batch(split: str='train'):
    '''
    generate a small batch of data of inputs x and targets y

    split: str: train or test
    '''
    data = train_data if split=='train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size, )) ##-- random idx
    x = torch.stack([data[i: i+block_size] for i in ix])
    y = torch.stack([data[i+1: i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return (x, y)

@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out


class Head(nn.Module):
    ''' one head of self-attention '''

    def __init__(self, head_size):
        super().__init__()
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

    def forward(self, x):
        B, T, C = x.shape
        q = self.query(x) # (B, T, C)
        k = self.key(x) # (B, T, C)
        ##-- compute attention score 'affinities' (взаимоотношения между токенами)
        wei = q @ k.transpose(-2, -1) * C**-0.5 # (B, T, C) @ (B, C, T) -->> (B, T, T)
        wei = wei.masked_fill(self.tril[:T, :T]==0, float('-inf')) # (B, T, T)
        wei = F.softmax(wei, dim=-1)
        # perform the weighted aggregation of the value
        v = self.value(x) # (B, T, C)
        out = wei @ v  # (B, T, T) @ (B, T, C) -->> (B, T, C)
        return out


class MultiHeadAttention(nn.Module):
    ''' multiple heads of self-attention in parallel '''

    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(n_embd, n_embd) ##-- for residual connection

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1) ##-- объединение по каналам
        out = self.proj(out)
        return out


class FeedForward(nn.Module):
    ''' a simple linear layer followed by a non-linearity '''

    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd) ##-- for residual connection
        )

    def forward(self, x):
        return self.net(x)


class Block(nn.Module):
    ''' Transformer block: communication followed by computation '''

    def __init__(self, n_embd, n_head):
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedForward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd) ##-- Layer Normalization by token (aka embedding)
        self.ln2 = nn.LayerNorm(n_embd) ##-- Layer Normalization by token (aka embedding)
    
    def forward(self, x):
        x = x + self.sa(self.ln1(x)) ##-- add residual connection ##-- and layer norm token
        x = x + self.ffwd(self.ln2(x)) ##-- add residual connection  ##-- and layer norm token
        return x


##-- super simple bigram model
class BigramLanguageModel(nn.Module):
    def __init__(self):
        super().__init__()
        ##-- each token directly read oof the logits for the next token from a lookip table
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd) ##-- берем эмбединг токена (кодируем токен)
        self.position_embedding_table = nn.Embedding(block_size, n_embd) ##-- кодируем позицию токена
        self.blocks = nn.Sequential(
            Block(n_embd, n_head=4),
            Block(n_embd, n_head=4),
            Block(n_embd, n_head=4),
            nn.LayerNorm(n_embd),
        ) ##-- MultiheadAttention -> FFN 3 times
        self.lm_head = nn.Linear(n_embd, vocab_size) ##-- linear_model head

    def forward(self, idx, targets=None):
        B, T = idx.shape

        ##-- idx and targets are both (B, T) tensor of integers
        tok_emb = self.token_embedding_table(idx) # (B, T, C) #-- (4, 8, 32), (batch_size, block_size, emb_size)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device)) # (T, C) (8, 32) ##-- кодируем позицию токена
        x = tok_emb + pos_emb ##-- (B, T, C) эмбединг токена и его позиции
        x = self.blocks(x) ##-- apply 4 heads of self-attention (B, T, C)
        logits = self.lm_head(x) # (B, T, vocab_size) #-- (4, 8, 65), (batch_size, block_size, vocab_size)
        # print(logits.shape) # (4, 8, 65)
        # print(targets.shape) # (4, 8)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            ##-- из батча размером 4 предложения переводим в одну строку (то есть четыре предложения в одно)
            logits = logits.view(B*T, C)
            ##-- второй вариант, таргеты в таком случае тоже не менять
            # logits = torch.permute(logits, (0, 2, 1))
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
        return logits, loss

    def generate(self, idx, max_new_tokens: int=100):
        ##-- idx is the (B, T) array of the indices in the current context
        for _ in range(max_new_tokens):
            ##-- crop idx to the last block_size tokens
            idx_cond = idx[:, -block_size:]
            ##-- get the predictions
            logits, _ = self(idx_cond)
            ##-- focus only on the last time step (последнюю букву)
            # print(f"logits_before ={logits}, {logits.shape}")
            logits = logits[:, -1, :] # becomes (B, C)
            # print(f"logits_after ={logits}, {logits.shape}")
            ##-- softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # print(probs)
            ##-- sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # print(idx_next)
            ##-- append sampled idx to the running sample
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

model = BigramLanguageModel()
m = model.to(device)

##-- pytorch optimizer
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3)

##-- fit

for iter in range(max_iters):
    if iter % eval_interval == 0:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    ##-- sample a batch of data
    xb, yb = get_batch('train')

    ##-- evaluate the loss
    logits, loss = m(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

##-- generate the model
context = torch.zeros((1, 1), dtype=torch.long, device=device)
print(decode(m.generate(context, max_new_tokens=500)[0].tolist()))

"""

In [69]:
FILENAME = 'bigram_v8.py'
save(bigram_v8, FILENAME)

Saved file at: /content/bigram_v8.py


In [70]:
! diff bigram_v7.py bigram_v8.py

2,3d1
< 
< 
129a128,129
>         self.ln1 = nn.LayerNorm(n_embd) ##-- Layer Normalization by token (aka embedding)
>         self.ln2 = nn.LayerNorm(n_embd) ##-- Layer Normalization by token (aka embedding)
132,133c132,133
<         x = x + self.sa(x) ##-- add residual connection
<         x = x + self.ffwd(x) ##-- add residual connection
---
>         x = x + self.sa(self.ln1(x)) ##-- add residual connection ##-- and layer norm token
>         x = x + self.ffwd(self.ln2(x)) ##-- add residual connection  ##-- and layer norm token
147a148
>             nn.LayerNorm(n_embd),


In [71]:
%%time

! python bigram_v8.py

step 0: train loss 4.3090, val loss 4.3083
step 500: train loss 2.3978, val loss 2.3985
step 1000: train loss 2.2651, val loss 2.2666
step 1500: train loss 2.1693, val loss 2.1893
step 2000: train loss 2.1301, val loss 2.1659
step 2500: train loss 2.0830, val loss 2.1305
step 3000: train loss 2.0505, val loss 2.1223
step 3500: train loss 2.0406, val loss 2.0994
step 4000: train loss 2.0137, val loss 2.0933
step 4500: train loss 1.9909, val loss 2.0895

And they bridce.

STAULOLET:
KING Proke? you eyarth the the gatals!
You that us croce. Wardetlascauesswick, you,
Dot comzorous
Yours, to fitie bet milend liblees if ensent, will is that Grose,
Be the now on you muselfinnd teal.
-hus wouch by prup; and plow you lord.
In amopeselaves
Momery would thake on in on her piibse to they srive cenchimed though
And strume kind thrupted so;
Angied must with ale of whith Prive my of that but
Hartioblisa
Sadaad the Ereford:
Go cour aar tey vry to chan you!
My
CPU times: user 660 ms, sys: 114 ms, total

# Scale up

Улучшение параметров модели

In [72]:
#@title bigram_v9
bigram_v9 = \
"""
import torch
import torch.nn as nn
from torch.nn import functional as F

##-- hyperparameters
batch_size = 64 ##-- сколько независимых объектов (предложений) обрабатывается за раз (параллельно)
block_size = 256 ##-- максимальная длина предложения (в нашем случае, количество символов)
max_iters = 5000
eval_interval = 500
learning_rate = 3e-4
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200
n_embd = 384 ##-- размер эмбединга
n_head = 6 ##-- количество голов в multihead attention
n_layer = 6 ##-- количество блоков
dropout = 0.2
##------------------

torch.manual_seed(1337)

# wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
with open('input.txt', 'r', encoding='utf-8') as file:
    text = file.read()

# here all the unique character that occur in this text
chars = sorted(list(set(text)))
vocab_size = len(chars)
##-- create mapping from characters to integers
stoi = { ch: i for i, ch in enumerate(chars) }
atoi = { i: ch for i, ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s]
decode = lambda l: ''.join([atoi[i] for i in l])

##-- Train and test splits
data = torch.tensor(encode(text), dtype=torch.long)
n = int(0.9 * len(data))
train_data = data[:n]
val_data = data[n:]

##-- data loading
def get_batch(split: str='train'):
    '''
    generate a small batch of data of inputs x and targets y

    split: str: train or test
    '''
    data = train_data if split=='train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size, )) ##-- random idx
    x = torch.stack([data[i: i+block_size] for i in ix])
    y = torch.stack([data[i+1: i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return (x, y)

@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out


class Head(nn.Module):
    ''' one head of self-attention '''

    def __init__(self, head_size):
        super().__init__()
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        B, T, C = x.shape
        q = self.query(x) # (B, T, C)
        k = self.key(x) # (B, T, C)
        ##-- compute attention score 'affinities' (взаимоотношения между токенами)
        wei = q @ k.transpose(-2, -1) * C**-0.5 # (B, T, C) @ (B, C, T) -->> (B, T, T)
        wei = wei.masked_fill(self.tril[:T, :T]==0, float('-inf')) # (B, T, T)
        wei = F.softmax(wei, dim=-1)
        wei = self.dropout(wei) ##-- некоторые токены не будут взаимодействовать друг с другом
        # perform the weighted aggregation of the value
        v = self.value(x) # (B, T, C)
        out = wei @ v  # (B, T, T) @ (B, T, C) -->> (B, T, C)
        return out


class MultiHeadAttention(nn.Module):
    ''' multiple heads of self-attention in parallel '''

    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(n_embd, n_embd) ##-- for residual connection
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1) ##-- объединение по каналам
        out = self.proj(out)
        out = self.dropout(out)
        return out


class FeedForward(nn.Module):
    ''' a simple linear layer followed by a non-linearity '''

    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd), ##-- for residual connection
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)


class Block(nn.Module):
    ''' Transformer block: communication followed by computation '''

    def __init__(self, n_embd, n_head):
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedForward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd) ##-- Layer Normalization by token (aka embedding)
        self.ln2 = nn.LayerNorm(n_embd) ##-- Layer Normalization by token (aka embedding)
    
    def forward(self, x):
        x = x + self.sa(self.ln1(x)) ##-- add residual connection ##-- and layer norm token
        x = x + self.ffwd(self.ln2(x)) ##-- add residual connection  ##-- and layer norm token
        return x


##-- super simple bigram model
class BigramLanguageModel(nn.Module):
    def __init__(self):
        super().__init__()
        ##-- each token directly read oof the logits for the next token from a lookip table
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd) ##-- берем эмбединг токена (кодируем токен)
        self.position_embedding_table = nn.Embedding(block_size, n_embd) ##-- кодируем позицию токена
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)]) ##-- MHA and FFWN n_layer times
        self.ln_f = nn.LayerNorm(n_embd) ##-- final layer norm
        self.lm_head = nn.Linear(n_embd, vocab_size) ##-- linear_model head

    def forward(self, idx, targets=None):
        B, T = idx.shape

        ##-- idx and targets are both (B, T) tensor of integers
        tok_emb = self.token_embedding_table(idx) # (B, T, C) #-- (4, 8, 32), (batch_size, block_size, emb_size)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device)) # (T, C) (8, 32) ##-- кодируем позицию токена
        x = tok_emb + pos_emb ##-- (B, T, C) эмбединг токена и его позиции
        x = self.blocks(x) ##-- apply 4 heads of self-attention (B, T, C)
        x = self.ln_f(x) # (B, T, C)
        logits = self.lm_head(x) # (B, T, vocab_size) #-- (4, 8, 65), (batch_size, block_size, vocab_size)
        # print(logits.shape) # (4, 8, 65)
        # print(targets.shape) # (4, 8)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            ##-- из батча размером 4 предложения переводим в одну строку (то есть четыре предложения в одно)
            logits = logits.view(B*T, C)
            ##-- второй вариант, таргеты в таком случае тоже не менять
            # logits = torch.permute(logits, (0, 2, 1))
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
        return logits, loss

    def generate(self, idx, max_new_tokens: int=100):
        ##-- idx is the (B, T) array of the indices in the current context
        for _ in range(max_new_tokens):
            ##-- crop idx to the last block_size tokens
            idx_cond = idx[:, -block_size:]
            ##-- get the predictions
            logits, _ = self(idx_cond)
            ##-- focus only on the last time step (последнюю букву)
            # print(f"logits_before ={logits}, {logits.shape}")
            logits = logits[:, -1, :] # becomes (B, C)
            # print(f"logits_after ={logits}, {logits.shape}")
            ##-- softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # print(probs)
            ##-- sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # print(idx_next)
            ##-- append sampled idx to the running sample
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

model = BigramLanguageModel()
m = model.to(device)

##-- pytorch optimizer
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3)

##-- fit

for iter in range(max_iters):
    if iter % eval_interval == 0:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    ##-- sample a batch of data
    xb, yb = get_batch('train')

    ##-- evaluate the loss
    logits, loss = m(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

##-- generate the model
context = torch.zeros((1, 1), dtype=torch.long, device=device)
print(decode(m.generate(context, max_new_tokens=500)[0].tolist()))

"""

In [73]:
FILENAME = 'bigram_v9.py'
save(bigram_v9, FILENAME)

Saved file at: /content/bigram_v9.py


In [74]:
! diff bigram_v8.py bigram_v9.py

7,8c7,8
< batch_size = 32 ##-- сколько независимых объектов (предложений) обрабатывается за раз (параллельно)
< block_size = 8 ##-- максимальная длина предложения (в нашем случае, количество символов)
---
> batch_size = 64 ##-- сколько независимых объектов (предложений) обрабатывается за раз (параллельно)
> block_size = 256 ##-- максимальная длина предложения (в нашем случае, количество символов)
11c11
< learning_rate = 1e-3
---
> learning_rate = 3e-4
14c14,17
< n_embd = 32 ##-- размер эмбединга
---
> n_embd = 384 ##-- размер эмбединга
> n_head = 6 ##-- количество голов в multihead attention
> n_layer = 6 ##-- количество блоков
> dropout = 0.2
76a80,81
>         self.dropout = nn.Dropout(dropout)
> 
84a90
>         wei = self.dropout(wei) ##-- некоторые токены не будут взаимодействовать друг с другом
97a104
>         self.dropout = nn.Dropout(dropout)
101a109
>         out = self.dropout(out)
113c121,122
<             nn.Linear(4 * n_embd, n_embd) ##-- for residual connection
---
>    

In [75]:
%%time

! python bigram_v9.py

step 0: train loss 4.2849, val loss 4.2824
step 500: train loss 1.6788, val loss 1.8442
step 1000: train loss 1.3603, val loss 1.5854
step 1500: train loss 1.2405, val loss 1.5051
step 2000: train loss 1.1581, val loss 1.4841
step 2500: train loss 1.0912, val loss 1.4882
step 3000: train loss 1.0312, val loss 1.4934
step 3500: train loss 0.9698, val loss 1.5189
step 4000: train loss 0.9121, val loss 1.5430
step 4500: train loss 0.8506, val loss 1.5726

Good night! How like to him?

GLOUCESTER:
Bress them for their father's moistress apace?
Alack, haste their within tender pines
I tread upon thee! Tread such a sad stars,
That, when the tiger new shades metimes, ere now
show nine to triumphs: but stinato, bawd?
Past noise, to quench:
All then all moons martings, were unto me,
Thou and mere at fierces or own to love.

KING RICHARD II:
Lay aside in banishhood stark what now haste?

QUEEN ELIZABETH:
Go, go, bear me; I will bear read till lo.
I mean
CPU times: user 4.92 s, sys: 776 ms, total